# <font color=BLUE size=10> 7ª Competição FLAI de 
# <font color=BLUE size=12> MACHINE LEARNING
    
**Author**: [Camila Maestrelli](https://www.linkedin.com/in/camila-maestrelli-leobons/)

***

# <font> IMPORTANDO AS BIBLIOTECAS

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 100)

# visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import sweetviz as sv

# ANOVA
!pip install statsmodels
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import scipy.stats as stats

# Machine Learning library
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import VotingRegressor

!pip install lightgbm
from lightgbm.sklearn import LGBMRegressor
!pip install xgboost
from xgboost import XGBRegressor

from sklearn.model_selection import cross_validate, RepeatedKFold, KFold, cross_val_predict
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline


from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler # used when there are many outliers
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import FunctionTransformer

!pip install category_encoders
import category_encoders as ce   

from sklearn.impute import SimpleImputer, KNNImputer      
from sklearn.compose import ColumnTransformer
from sklearn.compose import TransformedTargetRegressor

#metrics
from sklearn.metrics import mean_absolute_error , r2_score, mean_squared_error

import warnings
from datetime import datetime

# DEFININDO UMA FUNÇÃO PARA COMPARAÇÃO DOS MODELOS

A função nesta etapa definida apresenta um desvio padrão para que se tenha uma noção do intervalo que o EMA poderia assumir, ou seja o intervalo de confiança que o EMA pode expressar. Com 95% de confiança, EMA = (média - 2*std, média + 2*std)

In [3]:
# Useful function

# Function to compare models using cross validation
def cross_validating_compare(X, y, models, nomes, k=10, repeats=1, random_st = None ):
    
    validation = RepeatedKFold(n_splits=k, n_repeats = repeats, random_state = random_st)
    
    train_mean_list = []
    train_std_list = []
    test_mean_list =[]
    test_std_list =[]
    
    for model in models:
        scores = cross_validate(model, X, y, cv = validation, 
                                scoring=('neg_mean_absolute_error'),
                                return_train_score=True)
        
        mae_train = abs(scores['train_score'].mean().round(2))
        mae_train_std = scores['train_score'].std().round(2)
        mae_test = abs(scores['test_score'].mean().round(2))
        mae_test_std = scores['test_score'].std().round(2)
        
        
        train_mean_list.append(mae_train)
        train_std_list.append(mae_train_std)
        test_mean_list.append(mae_test)
        test_std_list.append(mae_test_std)
        
        
    aux = {'MAE TRAIN': train_mean_list, 'MAE TRAIN STD': train_std_list, 'MAE TEST': test_mean_list, 'MAE TEST STD': test_std_list}

    resultados = pd.DataFrame(aux, index = nomes)
    
    return resultados.sort_values(by = 'MAE TEST') 
        

# IMPORTANDO OS DADOS

In [5]:
# urls containing the datasets
train_url = 'https://raw.githubusercontent.com/camilamaestrelli/Wage-Prediction-a-Regression-Competition/master/treino.csv'
test_url = 'https://raw.githubusercontent.com/camilamaestrelli/Wage-Prediction-a-Regression-Competition/master/teste.csv'

# importing dataset
original_train = pd.read_csv(train_url)
original_test = pd.read_csv(test_url)

# copying the datsets
df_train = original_train.copy()
df_test = original_test.copy()

# PROCURANDO POR DADOS DUPLICADOS

In [6]:
# Searching for duplicates
df = pd.concat([df_train, df_test])
indices_duplicados = df.id.value_counts()[df.id.value_counts()>1].index

frames = []
for i in indices_duplicados:
    frames.append(df[df.id == i])

# DataFrame containing the duplicated data  
df_duplicado = pd.concat(frames)

In [7]:
# Train DataFrame containing the non-duplicates
df_train_nao_dup = df_train[~df_train.id.isin(indices_duplicados)]

# Test DataFrame containing the non-duplicates
df_test_nao_dup = df_test[~df_test.id.isin(indices_duplicados)]


# DEFININDO UM ESTIMADOR BASE PARA COMPARAÇÃO DAS ALTERAÇÕES

#### OBS.: Este modelo foi definido como base por ser o melhor modelo na 2ª submissão

In [8]:
# defining a regressor to compare possible transformations
modelo_base = GradientBoostingRegressor(n_estimators = 50, 
                                   min_samples_split = 7, 
                                   min_samples_leaf = 4, 
                                   max_features = 'log2', 
                                   max_depth = 12, 
                                   loss = 'lad', # The loss ‘lad’ was deprecated in v1.0 and will be removed in version 1.2. Use loss='absolute_error' which is equivalent.
                                   learning_rate = 0.1
                                        
                                       )

***

# PRÉ-PROCESSAMENTO: AVALIANDO AS POSSÍVEIS TRANSFORMAÇÕES 

## Funções para as transformações

#### As funções aqui definidas foram desenvolvidas a partir da EDA

In [9]:
# defining funstions for dataset transformations

def aux_no_change(df):
    return df


# feature: tempo_no_emprego
top_tempo_emprego = df_train['tempo_no_emprego'].quantile(0.975)
media_tempo_emprego = df_train['tempo_no_emprego'].mean()


def aux_emprego1(df):
    df.loc[df['tempo_no_emprego']>top_tempo_emprego] = top_tempo_emprego
    return df
 

#feature: horas_trabalho_por_semana
media_horas_semana = df_train['horas_trabalho_por_semana'].mean()
bottom_horas_semana = df_train['horas_trabalho_por_semana'].quantile(0.025)
    
def aux_horas_trab1(df):
    df.loc[df['horas_trabalho_por_semana']==0] = media_horas_semana
    return df

def aux_horas_trab2(df):
    df.loc[df['horas_trabalho_por_semana']<10] = media_horas_semana
    return df

def aux_horas_trab3(df):
    df.loc[df['horas_trabalho_por_semana']<bottom_horas_semana] = media_horas_semana
    return df

def aux_horas_trab4(df):
    df.loc[df['horas_trabalho_por_semana']<bottom_horas_semana] = bottom_horas_semana
    return df

def aux_horas_trabalho_zero(x): # In this case, x is a line value, not a DataFrame
    if x == 0:
        return 1
    else:
        return 0
    
def aux_add_horas_trab_zero(df):
    df['horas_trabalho_zero'] = df.horas_trabalho_por_semana.apply(aux_horas_trabalho_zero)
    return df
    
    
# feature: cor_raca
def aux_cor1(df):
    df.loc[df['cor_raca']==2]=6
    return df

def aux_cor2(df):
    df.loc[df['cor_raca']==2]=6
    df.loc[df['cor_raca']==9]=8
    return df

    
# feature: escolaridade
def aux_escolaridade1(df):
    for linha in df['escolaridade']:
        if linha <= 6 :
            linha = 1
        elif linha == 7:
            linha =2
        elif linha ==8:
            linha=3
        elif linha==9:
            linha=4
        else:
            linha=5
    return df

def aux_escolaridade2(df):
    for linha in df['escolaridade']:
        if linha < 5 :
            linha = 1
        elif linha < 7:
            linha =2
        elif linha < 9:
            linha=3
        elif linha==9:
            linha=4
        else:
            linha=5
    return df


# feature: tamanho_da_empresa
def aux_empresa1(df): 
    for linha in df['tamanho_da_empresa']:
        if linha == 10: 
            linha = 5
        elif linha >=8:
            linha = 4
        elif linha >=6:
            linha = 3
        elif linha >=4:
            linha = 2
        else:
            linha = 1
    return df
            
def aux_empresa2(df): 
    for linha in df['tamanho_da_empresa']:
        if linha >=9:
            linha = 4
        elif linha >=7:
            linha = 3
        elif linha >=5:
            linha = 2
        else:
            linha = 1
    return df


# feature: faixa_etaria
def aux_faixa_etaria(df):
    df.loc[df['faixa_etaria']==1]=2
    return df


# feature:sexo
def aux_sexo(df):
    df.loc[df['sexo']==2] = 0
    return df                


## Definindo as possíveis transformações 

In [10]:

# Numerical transformations
max_abs = MaxAbsScaler()
sc = StandardScaler()
min_max = MinMaxScaler()
robust = RobustScaler()
norma = Normalizer()
quanti_uni = QuantileTransformer(output_distribution='uniform')
quanti_norm = QuantileTransformer(output_distribution='normal')
p_trans = PowerTransformer(method = 'box-cox')
log1 = FunctionTransformer(np.log1p)
log = FunctionTransformer(np.log)

# Categorical encoders
bi_encoder = ce.BinaryEncoder()
target_encoder = ce.TargetEncoder()
out_encoder = ce.LeaveOneOutEncoder()

# My transformation functions
no_change = FunctionTransformer(aux_no_change)

employ = FunctionTransformer(aux_emprego1)

horas_trab1 = FunctionTransformer(aux_horas_trab1)
horas_trab2 = FunctionTransformer(aux_horas_trab2)
horas_trab3 = FunctionTransformer(aux_horas_trab3)
horas_trab4 = FunctionTransformer(aux_horas_trab4)
horas_zero = FunctionTransformer(aux_add_horas_trab_zero)

cor1 = FunctionTransformer(aux_cor1)
cor2 = FunctionTransformer(aux_cor2)

escolar1 = FunctionTransformer(aux_escolaridade1)
escolar2 = FunctionTransformer(aux_escolaridade2)

empresa1 = FunctionTransformer(aux_empresa1)
empresa2 = FunctionTransformer(aux_empresa2)

sexo = FunctionTransformer(aux_sexo)

f_eta = FunctionTransformer(aux_faixa_etaria)

# imputer
horas_imputer = KNNImputer(missing_values = 0, weights = 'distance', n_neighbors=2)

# Transformations for the numerical data
num_trans = [max_abs, sc, min_max, robust, quanti_uni, quanti_norm, log1, no_change]

# Tranformations for the categorical data (including the possibility of being treated as numerical)
cat_num_trans =  [max_abs, sc, min_max, robust, norma, log1, no_change, out_encoder, target_encoder, bi_encoder]



## Definindo o pipeline para cada variável

In [11]:
# defining the pipeline for each variable
tempo_pipeline = Pipeline(
    steps = [
        ("trans1", employ),
        ("scaler", robust),
        ])

horas_pipeline = Pipeline(
    steps = [
        ('zeradas', horas_zero),
        ("trans2", horas_trab1),
        ("scaler", robust),
        ])

cor_pipeline = Pipeline(
    steps = [
        ("trans3", cor1),
        ("scaler", robust),
        ])

escolar_pipeline = Pipeline(
    steps = [
        ("trans4", escolar1),
        ("scaler", robust),
        ])

empresa_pipeline = Pipeline(
    steps = [
        ("trans5", empresa1),
        ("scaler", robust),
        ])

f_etaria_pipeline = Pipeline(
    steps = [
        ("trans6", f_eta),
        ("scaler", robust),
        ])


# Setting up the pipeline transformation for each column/ variable
preprocessor = ColumnTransformer([('tempo', tempo_pipeline, ['tempo_no_emprego']), 
                                 ('horas_trabalho', horas_pipeline, ['horas_trabalho_por_semana']), 
                                 ('cor', cor_pipeline , ['cor_raca']),
                                 ('escolar', escolar_pipeline,['escolaridade']),
                                 ('empresa', empresa_pipeline, ['tamanho_da_empresa']),
                                 ('f_etaria', f_etaria_pipeline, ['faixa_etaria']),
                                 ('sex', sexo, ['sexo'])
                          
                                 ],
                                remainder='passthrough')

## Definindo o modelo agregado e as possíveis transformações para o RandomizedSearchCV

In [12]:
pipe1 = Pipeline(steps=[("preprocessor", preprocessor),('GradBoost', modelo_base)])

pipe = TransformedTargetRegressor(regressor=pipe1,
                                   func=np.log, 
                                   inverse_func=np.exp)



In [ ]:
param_grid = [{
    'regressor__preprocessor__tempo__trans1': [employ, no_change],               # tempo_no_emprego
    'regressor__preprocessor__tempo__scaler': num_trans,           
     
    'regressor__preprocessor__horas_trabalho__zeradas': [no_change, horas_zero],  # horas_trabalho_por_semana
    'regressor__preprocessor__horas_trabalho__trans2': [horas_trab1, horas_trab2, horas_trab3, horas_trab4, no_change, horas_imputer],
    'regressor__preprocessor__horas_trabalho__scaler': num_trans,
    
    'regressor__preprocessor__cor__trans3': [cor1, cor2, no_change],             # cor_raca
    'regressor__preprocessor__cor__scaler': cat_num_trans,
    
    'regressor__preprocessor__escolar__trans4':[escolar1, escolar2, no_change], # escolaridade
    'regressor__preprocessor__escolar__scaler':cat_num_trans,
    
    'regressor__preprocessor__empresa__trans5':[empresa1, empresa2, no_change],   # tamanho_empresa
    'regressor__preprocessor__empresa__scaler':cat_num_trans,
    
    'regressor__preprocessor__f_etaria__trans6':[f_eta, no_change],              # faixa_etaria
    'regressor__preprocessor__f_etaria__scaler':cat_num_trans,
    
    'regressor__preprocessor__sex': [sexo, no_change]                           # sexo
    
    }]

## Buscando as melhores transformações usando o RandomizedSearchCV

#### Como eram muitas possibilidades de transformações, o Grid Search se mostrou inviável para este projeto

In [37]:
    
X = df_train.drop(columns = ['remuneracao', 'id', 'idade'], axis = 1)
y =df_train['remuneracao']
    
validation = RepeatedKFold(n_splits=8, n_repeats = 1, random_state = 1)

grid_search_pipe = RandomizedSearchCV(pipe, param_grid, cv=validation,
                                scoring='neg_mean_absolute_error', verbose=2,
                                     n_iter=50)
grid_search_pipe.fit(X, y)

Fitting 8 folds for each of 50 candidates, totalling 400 fits
[CV] END regressor__preprocessor__cor__scaler=LeaveOneOutEncoder(), regressor__preprocessor__cor__trans3=FunctionTransformer(func=<function aux_no_change at 0x0000012F43D0CEE0>), regressor__preprocessor__empresa__scaler=MinMaxScaler(), regressor__preprocessor__empresa__trans5=FunctionTransformer(func=<function aux_empresa1 at 0x0000012F43878DC0>), regressor__preprocessor__escolar__scaler=StandardScaler(), regressor__preprocessor__escolar__trans4=FunctionTransformer(func=<function aux_escolaridade2 at 0x0000012F43D28040>), regressor__preprocessor__f_etaria__scaler=LeaveOneOutEncoder(), regressor__preprocessor__f_etaria__trans6=FunctionTransformer(func=<function aux_no_change at 0x0000012F43D0CEE0>), regressor__preprocessor__horas_trabalho__trans2=FunctionTransformer(func=<function aux_no_change at 0x0000012F43D0CEE0>), regressor__preprocessor__horas_trabalho__zeradas=FunctionTransformer(func=<function aux_add_horas_trab_zero 

[CV] END regressor__preprocessor__cor__scaler=LeaveOneOutEncoder(), regressor__preprocessor__cor__trans3=FunctionTransformer(func=<function aux_no_change at 0x0000012F43D0CEE0>), regressor__preprocessor__empresa__scaler=MinMaxScaler(), regressor__preprocessor__empresa__trans5=FunctionTransformer(func=<function aux_empresa1 at 0x0000012F43878DC0>), regressor__preprocessor__escolar__scaler=StandardScaler(), regressor__preprocessor__escolar__trans4=FunctionTransformer(func=<function aux_escolaridade2 at 0x0000012F43D28040>), regressor__preprocessor__f_etaria__scaler=LeaveOneOutEncoder(), regressor__preprocessor__f_etaria__trans6=FunctionTransformer(func=<function aux_no_change at 0x0000012F43D0CEE0>), regressor__preprocessor__horas_trabalho__trans2=FunctionTransformer(func=<function aux_no_change at 0x0000012F43D0CEE0>), regressor__preprocessor__horas_trabalho__zeradas=FunctionTransformer(func=<function aux_add_horas_trab_zero at 0x0000012F4377EA60>), regressor__preprocessor__sex=Function

[CV] END regressor__preprocessor__cor__scaler=StandardScaler(), regressor__preprocessor__cor__trans3=FunctionTransformer(func=<function aux_cor1 at 0x0000012F43D28AF0>), regressor__preprocessor__empresa__scaler=LeaveOneOutEncoder(), regressor__preprocessor__empresa__trans5=FunctionTransformer(func=<function aux_no_change at 0x0000012F43D0CEE0>), regressor__preprocessor__escolar__scaler=LeaveOneOutEncoder(), regressor__preprocessor__escolar__trans4=FunctionTransformer(func=<function aux_escolaridade1 at 0x0000012F43D283A0>), regressor__preprocessor__f_etaria__scaler=MaxAbsScaler(), regressor__preprocessor__f_etaria__trans6=FunctionTransformer(func=<function aux_no_change at 0x0000012F43D0CEE0>), regressor__preprocessor__horas_trabalho__trans2=FunctionTransformer(func=<function aux_no_change at 0x0000012F43D0CEE0>), regressor__preprocessor__horas_trabalho__zeradas=FunctionTransformer(func=<function aux_no_change at 0x0000012F43D0CEE0>), regressor__preprocessor__sex=FunctionTransformer(fu

[CV] END regressor__preprocessor__cor__scaler=Normalizer(), regressor__preprocessor__cor__trans3=FunctionTransformer(func=<function aux_cor1 at 0x0000012F43D28AF0>), regressor__preprocessor__empresa__scaler=Normalizer(), regressor__preprocessor__empresa__trans5=FunctionTransformer(func=<function aux_empresa2 at 0x0000012F43878310>), regressor__preprocessor__escolar__scaler=MaxAbsScaler(), regressor__preprocessor__escolar__trans4=FunctionTransformer(func=<function aux_escolaridade2 at 0x0000012F43D28040>), regressor__preprocessor__f_etaria__scaler=TargetEncoder(), regressor__preprocessor__f_etaria__trans6=FunctionTransformer(func=<function aux_no_change at 0x0000012F43D0CEE0>), regressor__preprocessor__horas_trabalho__trans2=FunctionTransformer(func=<function aux_horas_trab3 at 0x0000012F43D28C10>), regressor__preprocessor__horas_trabalho__zeradas=FunctionTransformer(func=<function aux_no_change at 0x0000012F43D0CEE0>), regressor__preprocessor__sex=FunctionTransformer(func=<function aux

[CV] END regressor__preprocessor__cor__scaler=MaxAbsScaler(), regressor__preprocessor__cor__trans3=FunctionTransformer(func=<function aux_cor2 at 0x0000012F43D28280>), regressor__preprocessor__empresa__scaler=StandardScaler(), regressor__preprocessor__empresa__trans5=FunctionTransformer(func=<function aux_empresa1 at 0x0000012F43878DC0>), regressor__preprocessor__escolar__scaler=FunctionTransformer(func=<ufunc 'log1p'>), regressor__preprocessor__escolar__trans4=FunctionTransformer(func=<function aux_escolaridade2 at 0x0000012F43D28040>), regressor__preprocessor__f_etaria__scaler=LeaveOneOutEncoder(), regressor__preprocessor__f_etaria__trans6=FunctionTransformer(func=<function aux_faixa_etaria at 0x0000012F43DA3040>), regressor__preprocessor__horas_trabalho__trans2=FunctionTransformer(func=<function aux_horas_trab2 at 0x0000012F43D28160>), regressor__preprocessor__horas_trabalho__zeradas=FunctionTransformer(func=<function aux_no_change at 0x0000012F43D0CEE0>), regressor__preprocessor__s

...

[CV] END regressor__preprocessor__cor__scaler=Normalizer(), regressor__preprocessor__cor__trans3=FunctionTransformer(func=<function aux_cor2 at 0x0000012F43D28280>), regressor__preprocessor__empresa__scaler=LeaveOneOutEncoder(), regressor__preprocessor__empresa__trans5=FunctionTransformer(func=<function aux_empresa2 at 0x0000012F43878310>), regressor__preprocessor__escolar__scaler=MinMaxScaler(), regressor__preprocessor__escolar__trans4=FunctionTransformer(func=<function aux_no_change at 0x0000012F43D0CEE0>), regressor__preprocessor__f_etaria__scaler=FunctionTransformer(func=<function aux_no_change at 0x0000012F43D0CEE0>), regressor__preprocessor__f_etaria__trans6=FunctionTransformer(func=<function aux_no_change at 0x0000012F43D0CEE0>), regressor__preprocessor__horas_trabalho__trans2=FunctionTransformer(func=<function aux_horas_trab2 at 0x0000012F43D28160>), regressor__preprocessor__horas_trabalho__zeradas=FunctionTransformer(func=<function aux_add_horas_trab_zero at 0x0000012F4377EA60

[CV] END regressor__preprocessor__cor__scaler=FunctionTransformer(func=<function aux_no_change at 0x0000012F43D0CEE0>), regressor__preprocessor__cor__trans3=FunctionTransformer(func=<function aux_cor1 at 0x0000012F43D28AF0>), regressor__preprocessor__empresa__scaler=TargetEncoder(), regressor__preprocessor__empresa__trans5=FunctionTransformer(func=<function aux_no_change at 0x0000012F43D0CEE0>), regressor__preprocessor__escolar__scaler=LeaveOneOutEncoder(), regressor__preprocessor__escolar__trans4=FunctionTransformer(func=<function aux_escolaridade1 at 0x0000012F43D283A0>), regressor__preprocessor__f_etaria__scaler=TargetEncoder(), regressor__preprocessor__f_etaria__trans6=FunctionTransformer(func=<function aux_no_change at 0x0000012F43D0CEE0>), regressor__preprocessor__horas_trabalho__trans2=FunctionTransformer(func=<function aux_horas_trab4 at 0x0000012F43D28A60>), regressor__preprocessor__horas_trabalho__zeradas=FunctionTransformer(func=<function aux_add_horas_trab_zero at 0x0000012

[CV] END regressor__preprocessor__cor__scaler=Normalizer(), regressor__preprocessor__cor__trans3=FunctionTransformer(func=<function aux_no_change at 0x0000012F43D0CEE0>), regressor__preprocessor__empresa__scaler=BinaryEncoder(), regressor__preprocessor__empresa__trans5=FunctionTransformer(func=<function aux_empresa1 at 0x0000012F43878DC0>), regressor__preprocessor__escolar__scaler=BinaryEncoder(), regressor__preprocessor__escolar__trans4=FunctionTransformer(func=<function aux_no_change at 0x0000012F43D0CEE0>), regressor__preprocessor__f_etaria__scaler=FunctionTransformer(func=<function aux_no_change at 0x0000012F43D0CEE0>), regressor__preprocessor__f_etaria__trans6=FunctionTransformer(func=<function aux_faixa_etaria at 0x0000012F43DA3040>), regressor__preprocessor__horas_trabalho__trans2=KNNImputer(missing_values=0, n_neighbors=2, weights='distance'), regressor__preprocessor__horas_trabalho__zeradas=FunctionTransformer(func=<function aux_no_change at 0x0000012F43D0CEE0>), regressor__pr

[CV] END regressor__preprocessor__cor__scaler=FunctionTransformer(func=<ufunc 'log1p'>), regressor__preprocessor__cor__trans3=FunctionTransformer(func=<function aux_no_change at 0x0000012F43D0CEE0>), regressor__preprocessor__empresa__scaler=StandardScaler(), regressor__preprocessor__empresa__trans5=FunctionTransformer(func=<function aux_no_change at 0x0000012F43D0CEE0>), regressor__preprocessor__escolar__scaler=MaxAbsScaler(), regressor__preprocessor__escolar__trans4=FunctionTransformer(func=<function aux_escolaridade1 at 0x0000012F43D283A0>), regressor__preprocessor__f_etaria__scaler=MinMaxScaler(), regressor__preprocessor__f_etaria__trans6=FunctionTransformer(func=<function aux_faixa_etaria at 0x0000012F43DA3040>), regressor__preprocessor__horas_trabalho__trans2=FunctionTransformer(func=<function aux_horas_trab3 at 0x0000012F43D28C10>), regressor__preprocessor__horas_trabalho__zeradas=FunctionTransformer(func=<function aux_add_horas_trab_zero at 0x0000012F4377EA60>), regressor__prepr

RandomizedSearchCV(cv=RepeatedKFold(n_repeats=1, n_splits=8, random_state=1),
                   estimator=TransformedTargetRegressor(func=<ufunc 'log'>,
                                                        inverse_func=<ufunc 'exp'>,
                                                        regressor=Pipeline(steps=[('preprocessor',
                                                                                   ColumnTransformer(remainder='passthrough',
                                                                                                     transformers=[('tempo',
                                                                                                                    Pipeline(steps=[('trans1',
                                                                                                                                     FunctionTransformer(func=<function aux_emprego1 at 0x0000012F43D...
                                         'regressor__preprocessor__s

In [39]:
resultados = pd.DataFrame(grid_search_pipe.cv_results_)
resultados.sort_values(by = 'rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__preprocessor__tempo__trans1,param_regressor__preprocessor__sex,param_regressor__preprocessor__horas_trabalho__zeradas,param_regressor__preprocessor__horas_trabalho__trans2,param_regressor__preprocessor__f_etaria__trans6,param_regressor__preprocessor__f_etaria__scaler,param_regressor__preprocessor__escolar__trans4,param_regressor__preprocessor__escolar__scaler,param_regressor__preprocessor__empresa__trans5,param_regressor__preprocessor__empresa__scaler,param_regressor__preprocessor__cor__trans3,param_regressor__preprocessor__cor__scaler,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,mean_test_score,std_test_score,rank_test_score
45,31.938203,0.304811,1.913756,0.062448,FunctionTransformer(func=<function aux_no_chan...,FunctionTransformer(func=<function aux_no_chan...,FunctionTransformer(func=<function aux_add_hor...,"KNNImputer(missing_values=0, n_neighbors=2, we...",FunctionTransformer(func=<function aux_no_chan...,FunctionTransformer(func=<ufunc 'log1p'>),FunctionTransformer(func=<function aux_no_chan...,MinMaxScaler(),FunctionTransformer(func=<function aux_empresa...,MinMaxScaler(),FunctionTransformer(func=<function aux_cor1 at...,FunctionTransformer(func=<function aux_no_chan...,{'regressor__preprocessor__tempo__trans1': Fun...,-1396.685879,-1209.385600,-1248.336386,-1337.991679,-1285.852636,-1242.417007,-1265.201047,-1275.898460,-1282.721087,55.442469,1
1,28.516206,3.122032,0.073396,0.012138,FunctionTransformer(func=<function aux_no_chan...,FunctionTransformer(func=<function aux_no_chan...,FunctionTransformer(func=<function aux_no_chan...,FunctionTransformer(func=<function aux_no_chan...,FunctionTransformer(func=<function aux_no_chan...,MaxAbsScaler(),FunctionTransformer(func=<function aux_escolar...,LeaveOneOutEncoder(),FunctionTransformer(func=<function aux_no_chan...,LeaveOneOutEncoder(),FunctionTransformer(func=<function aux_cor1 at...,StandardScaler(),{'regressor__preprocessor__tempo__trans1': Fun...,-1394.776074,-1215.140756,-1245.600261,-1345.133133,-1284.200181,-1248.470973,-1261.365210,-1268.571941,-1282.907316,55.031687,2
41,17.653534,0.808148,0.050863,0.004290,FunctionTransformer(func=<function aux_emprego...,FunctionTransformer(func=<function aux_no_chan...,FunctionTransformer(func=<function aux_add_hor...,FunctionTransformer(func=<function aux_no_chan...,FunctionTransformer(func=<function aux_faixa_e...,StandardScaler(),FunctionTransformer(func=<function aux_no_chan...,FunctionTransformer(func=<function aux_no_chan...,FunctionTransformer(func=<function aux_empresa...,FunctionTransformer(func=<ufunc 'log1p'>),FunctionTransformer(func=<function aux_cor2 at...,FunctionTransformer(func=<ufunc 'log1p'>),{'regressor__preprocessor__tempo__trans1': Fun...,-1407.646858,-1214.620552,-1246.986732,-1342.045929,-1277.995180,-1246.542312,-1263.239528,-1267.837972,-1283.364383,58.061904,3
8,17.726963,0.275392,0.060961,0.017464,FunctionTransformer(func=<function aux_emprego...,FunctionTransformer(func=<function aux_sexo at...,FunctionTransformer(func=<function aux_add_hor...,FunctionTransformer(func=<function aux_horas_t...,FunctionTransformer(func=<function aux_faixa_e...,LeaveOneOutEncoder(),FunctionTransformer(func=<function aux_escolar...,RobustScaler(),FunctionTransformer(func=<function aux_no_chan...,FunctionTransformer(func=<ufunc 'log1p'>),FunctionTransformer(func=<function aux_no_chan...,StandardScaler(),{'regressor__preprocessor__tempo__trans1': Fun...,-1403.602057,-1206.632222,-1245.879967,-1353.695949,-1285.117779,-1248.118066,-1259.178297,-1267.516889,-1283.717653,59.976060,4
4,17.253355,0.510173,0.057969,0.007906,FunctionTransformer(func=<function aux_no_chan...,FunctionTransformer(func=<function aux_sexo at...,FunctionTransformer(func=<function aux_add_hor...,FunctionTransformer(func=<function aux_no_chan...,FunctionTransformer(func=<f

In [ ]:

{'regressor__preprocessor__tempo__trans1': FunctionTransformer(func=<function aux_no_change at 0x0000012F43D0CEE0>),
 'regressor__preprocessor__tempo__scaler': FunctionTransformer(func=<ufunc 'log1p'>),
 'regressor__preprocessor__sex': FunctionTransformer(func=<function aux_no_change at 0x0000012F43D0CEE0>),
 'regressor__preprocessor__horas_trabalho__zeradas': FunctionTransformer(func=<function aux_add_horas_trab_zero at 0x0000012F4377EA60>),
 'regressor__preprocessor__horas_trabalho__trans2': FunctionTransformer(func=<function aux_horas_trab1 at 0x0000012F43D28E50>),
 'regressor__preprocessor__horas_trabalho__scaler': FunctionTransformer(func=<ufunc 'log1p'>),
 'regressor__preprocessor__f_etaria__trans6': FunctionTransformer(func=<function aux_faixa_etaria at 0x0000012F43DA3040>),
 'regressor__preprocessor__f_etaria__scaler': MinMaxScaler(),
 'regressor__preprocessor__escolar__trans4': FunctionTransformer(func=<function aux_no_change at 0x0000012F43D0CEE0>),
 'regressor__preprocessor__escolar__scaler': FunctionTransformer(func=<function aux_no_change at 0x0000012F43D0CEE0>),
 'regressor__preprocessor__empresa__trans5': FunctionTransformer(func=<function aux_empresa1 at 0x0000012F43878DC0>),
 'regressor__preprocessor__empresa__scaler': RobustScaler(),
 'regressor__preprocessor__cor__trans3': FunctionTransformer(func=<function aux_no_change at 0x0000012F43D0CEE0>),
 'regressor__preprocessor__cor__scaler': MinMaxScaler()}
1279


***

# <font color=BLUE> ADOTANDO O MELHOR RESULTADO CONFORME O RESULTADO DO RANDOMIZEDSEARCHCV

In [13]:
tempo_pipeline = Pipeline(
    steps = [
        ("trans1", no_change),
        ("scaler", log1),
        ])

horas_pipeline = Pipeline(
    steps = [
        ('zeradas', horas_zero),
        ("trans2", horas_trab1),
        ("scaler", log1),
        ])

cor_pipeline = Pipeline(
    steps = [
        ("trans3", no_change),
        ("scaler", min_max),
        ])

escolar_pipeline = Pipeline(
    steps = [
        ("trans4", no_change),
        ("scaler", no_change),
        ])

empresa_pipeline = Pipeline(
    steps = [
        ("trans5", empresa1),
        ("scaler", robust),
        ])

f_etaria_pipeline = Pipeline(
    steps = [
        ("trans6", f_eta),
        ("scaler", min_max),
        ])

preprocessor = ColumnTransformer([('tempo', tempo_pipeline, ['tempo_no_emprego']), 
                                 ('horas_trabalho', horas_pipeline, ['horas_trabalho_por_semana']), 
                                 ('cor', cor_pipeline , ['cor_raca']),
                                 ('escolar', escolar_pipeline,['escolaridade']),
                                 ('empresa', empresa_pipeline, ['tamanho_da_empresa']),
                                 ('f_etaria', f_etaria_pipeline, ['faixa_etaria']),
                                 ('sex', no_change, ['sexo'])
                          
                                 ],
                                remainder='passthrough')



# <font > BUSCANDO OS MELHORES ESTIMADORES PELO RANDOMIZED SEARCH CV

## Avaliando o XGBRegressor

In [49]:
modelo = XGBRegressor()

pipe1 = Pipeline(steps=[("preprocessor", preprocessor),('XGB', modelo)])

pipe = [TransformedTargetRegressor(regressor=pipe1,
                                   func=np.log, 
                                   inverse_func=np.exp)]

resultados = cross_validating_compare(X, y, pipe, ['log'], k=10, repeats=1, random_st = 1)
resultados

,MAE TRAIN,MAE TRAIN STD,MAE TEST,MAE TEST STD
log,1011.4,3.81,1307.77,50.73


In [50]:
modelo_em_teste = XGBRegressor()


parametros = {
                'regressor__modelo__booster': ['gbtree'],
                'regressor__modelo__eta': np.arange(0.01, 0.8, 0.05), 
                'regressor__modelo__max_depth': np.arange(2,20),
                'regressor__modelo__min_child_weight': np.arange(1,4),
                'regressor__modelo__subsample': np.arange(0.5, 1.05, 0.05),
                'regressor__modelo__base_score': np.arange(0.25, 1.05, 0.05),
                'regressor__modelo__n_estimators': np.arange(100, 2500, 100),
                'regressor__modelo__colsample_bynode': np.arange(0.5, 1.05, 0.05),
                'regressor__modelo__num_parallel_tree': np.arange(1,4)
                }


pipe1 = Pipeline(steps=[("preprocessor", preprocessor),('modelo', modelo_em_teste)])

pipe = TransformedTargetRegressor(regressor=pipe1,
                                   func=np.log, 
                                   inverse_func=np.exp)

validation = RepeatedKFold(n_splits=8, n_repeats = 1, random_state = 1)

grid_search_pipe = RandomizedSearchCV(pipe, parametros, cv=validation,
                                      scoring='neg_mean_absolute_error', verbose=2,
                                      return_train_score = True,
                                      n_iter=10,
                                      #n_jobs=-1
                                     )

grid_search_pipe.fit(X, y)

resultados = pd.DataFrame(grid_search_pipe.cv_results_)
resultados.sort_values(by = 'rank_test_score')

Fitting 8 folds for each of 10 candidates, totalling 80 fits
[CV] END regressor__modelo__base_score=0.7, regressor__modelo__booster=gbtree, regressor__modelo__colsample_bynode=0.6500000000000001, regressor__modelo__eta=0.36000000000000004, regressor__modelo__max_depth=9, regressor__modelo__min_child_weight=2, regressor__modelo__n_estimators=100, regressor__modelo__num_parallel_tree=1, regressor__modelo__subsample=0.9000000000000004; total time=  16.5s
[CV] END regressor__modelo__base_score=0.7, regressor__modelo__booster=gbtree, regressor__modelo__colsample_bynode=0.6500000000000001, regressor__modelo__eta=0.36000000000000004, regressor__modelo__max_depth=9, regressor__modelo__min_child_weight=2, regressor__modelo__n_estimators=100, regressor__modelo__num_parallel_tree=1, regressor__modelo__subsample=0.9000000000000004; total time=  12.8s
[CV] END regressor__modelo__base_score=0.7, regressor__modelo__booster=gbtree, regressor__modelo__colsample_bynode=0.6500000000000001, regressor__mod

[CV] END regressor__modelo__base_score=0.9999999999999998, regressor__modelo__booster=gbtree, regressor__modelo__colsample_bynode=0.55, regressor__modelo__eta=0.31000000000000005, regressor__modelo__max_depth=18, regressor__modelo__min_child_weight=1, regressor__modelo__n_estimators=1100, regressor__modelo__num_parallel_tree=1, regressor__modelo__subsample=0.6000000000000001; total time= 4.4min
[CV] END regressor__modelo__base_score=0.9999999999999998, regressor__modelo__booster=gbtree, regressor__modelo__colsample_bynode=0.55, regressor__modelo__eta=0.31000000000000005, regressor__modelo__max_depth=18, regressor__modelo__min_child_weight=1, regressor__modelo__n_estimators=1100, regressor__modelo__num_parallel_tree=1, regressor__modelo__subsample=0.6000000000000001; total time= 4.4min
[CV] END regressor__modelo__base_score=0.9999999999999998, regressor__modelo__booster=gbtree, regressor__modelo__colsample_bynode=0.55, regressor__modelo__eta=0.31000000000000005, regressor__modelo__max_d

[CV] END regressor__modelo__base_score=0.5999999999999999, regressor__modelo__booster=gbtree, regressor__modelo__colsample_bynode=0.9000000000000004, regressor__modelo__eta=0.76, regressor__modelo__max_depth=9, regressor__modelo__min_child_weight=1, regressor__modelo__n_estimators=1800, regressor__modelo__num_parallel_tree=2, regressor__modelo__subsample=0.6500000000000001; total time= 4.4min
[CV] END regressor__modelo__base_score=0.5999999999999999, regressor__modelo__booster=gbtree, regressor__modelo__colsample_bynode=0.9000000000000004, regressor__modelo__eta=0.76, regressor__modelo__max_depth=9, regressor__modelo__min_child_weight=1, regressor__modelo__n_estimators=1800, regressor__modelo__num_parallel_tree=2, regressor__modelo__subsample=0.6500000000000001; total time= 5.5min
[CV] END regressor__modelo__base_score=0.5999999999999999, regressor__modelo__booster=gbtree, regressor__modelo__colsample_bynode=0.9000000000000004, regressor__modelo__eta=0.76, regressor__modelo__max_depth=

[CV] END regressor__modelo__base_score=0.35, regressor__modelo__booster=gbtree, regressor__modelo__colsample_bynode=0.6000000000000001, regressor__modelo__eta=0.66, regressor__modelo__max_depth=6, regressor__modelo__min_child_weight=2, regressor__modelo__n_estimators=1700, regressor__modelo__num_parallel_tree=2, regressor__modelo__subsample=0.7500000000000002; total time= 3.9min
[CV] END regressor__modelo__base_score=0.7, regressor__modelo__booster=gbtree, regressor__modelo__colsample_bynode=0.7500000000000002, regressor__modelo__eta=0.16000000000000003, regressor__modelo__max_depth=19, regressor__modelo__min_child_weight=2, regressor__modelo__n_estimators=1900, regressor__modelo__num_parallel_tree=1, regressor__modelo__subsample=0.7500000000000002; total time=12.3min
[CV] END regressor__modelo__base_score=0.7, regressor__modelo__booster=gbtree, regressor__modelo__colsample_bynode=0.7500000000000002, regressor__modelo__eta=0.16000000000000003, regressor__modelo__max_depth=19, regressor

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__modelo__subsample,param_regressor__modelo__num_parallel_tree,param_regressor__modelo__n_estimators,param_regressor__modelo__min_child_weight,param_regressor__modelo__max_depth,param_regressor__modelo__eta,param_regressor__modelo__colsample_bynode,param_regressor__modelo__booster,param_regressor__modelo__base_score,params,split0_test_score,...,split6_test_score,split7_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,split5_train_score,split6_train_score,split7_train_score,mean_train_score,std_train_score
0,15.238494,1.885680,0.077543,0.033054,0.9,1,100,2,9,0.36,0.65,gbtree,0.7,{'regressor__modelo__subsample': 0.90000000000...,-1455.225942,...,-1314.309625,-1369.332004,-1378.579907,45.785596,1,-644.817846,-650.659387,-650.502185,-654.808120,-647.737571,-647.220734,-661.110014,-657.339402,-651.774407,5.191504
4,133.879804,25.184819,0.130402,0.023068,0.6,2,2400,2,3,0.41,0.9,gbtree,0.65,{'regressor__modelo__subsample': 0.60000000000...,-1472.053866,...,-1374.398083,-1352.676834,-1408.436328,56.224368,2,-844.723680,-857.242271,-852.018108,-845.835606,-846.503101,-853.920676,-850.256549,-860.432102,-851.366512,5.282763
6,666.308381,75.545926,0.675568,0.078613,0.85,3,700,2,19,0.26,0.7,gbtree,0.45,{'regressor__modelo__subsample': 0.85000000000...,-1525.638764,...,-1374.292271,-1459.617738,-1422.616482,57.938149,3,-40.817096,-46.232327,-47.671328,-44.573962,-46.720423,-45.977985,-44.565489,-44.383094,-45.117713,1.959198
9,215.081619,48.595957,0.182512,0.016954,0.7,3,2400,3,3,0.66,0.8,gbtree,0.3,{'regressor__modelo__subsample': 0.70000000000...,-1513.491183,...,-1407.625109,-1389.633934,-1436.288670,57.080635,4,-766.584887,-785.244326,-781.462335,-772.498039,-771.990978,-777.801920,-777.660912,-784.690177,-777.241697,6.138661
3,82.481839,42.520897,0.123420,0.022165,0.9,1,1700,2,6,0.21,0.5,gbtree,0.95,{'regressor__modelo__subsample': 0.90000000000...,-1506.544569,...,-1404.510747,-1418.684755,-1437.508441,46.115416,5,-423.379598,-425.793233,-426.219535,-430.207542,-419.363550,-426.477031,-426.197654,-433.341240,-426.372423,3.901188
8,743.360190,32.213761,0.616601,0.064724,0.75,1,1900,2,19,0.16,0.75,gbtree,0.7,{'regressor__modelo__subsample': 0.75000000000...,-1552.088155,...,-1386.092205,-1506.944793,-1457.556428,56.328122,6,-44.908118,-50.119742,-51.087412,-48.206523,-50.613904,-49.640258,-48.118640,-48.532983,-48.903447,1.834365
1,169.440391,55.543363,0.140250,0.025215,0.75,3,400,2,9,0.51,0.75,gbtree,0.6,{'regressor__modelo__subsample': 0.75000000000...,-1567.666189,...,-1455.178061,-1513.927341,-1486.783679,54.090500,7,-129.814288,-130.495150,-134.746465,-129.304280,-126.493622,-135.034582,-131.831765,-138.513524,-132.029210,3.602573
2,462.409307,190.775619,0.463760,0.077756,0.6,1,1100,1,18,0.31,0.55,gbtree,1.0,{'regressor__modelo__subsample': 0.60000000000...,-1693.932855,...,-1524.087010,-1613.203472,-1559.593376,65.681604,8,-54.858611,-60.334652,-61.252899,-58.613395,-59.347980,-59.591805,-58.072241,-57.209465,-58.660131,1.863176
7,233.323330,77.026210,0.181640,0.021796,0.75,2,1700,2,6,0.66,0.6,gbtree,0.35,{'regressor__modelo__subsample': 0.75000000000...,-1655.339877,...,-1514.094615,-1560.753604,-1571.564194,49.651939,9,-176.214458,-181.188629,-178.977627,-177.837086,-171.146685,-183.939422,-178.630046,-186.239348,-179.271663,4.357885
5,315.815594,66.295595,0.318171,0.028215,0.65,2,1800,1,9,0.76,0.9,gbtree,0.6,{'regressor__modelo__subsample': 0.65000000000...,-1775.280083,...,-1587.004558,-2047.024308,-1747.098945,143.252881,10,-72.062568,-76.810043,-79.044451,-74.143076,-76.249860,-77.403353,-74.500585,-75.032185,-75.655765,2.035471


In [26]:
outro_xgb = XGBRegressor(learning_rate = 0.01, 
                        n_estimators = 2200, 
                        subsample = 0.9000000000000004, 
                        colsample_bynode = 0.7000000000000002, 
                        objective = 'reg:squarederror',
                        base_score = 0.3,
                        num_parallel_tree = 1,
                        min_child_weight = 2,
                        max_depth = 4
                       )

ipe1 = Pipeline(steps=[("preprocessor", preprocessor),('modelo', outro_xgb)])

pipe = [TransformedTargetRegressor(regressor=pipe1,
                                   func=np.log, 
                                   inverse_func=np.exp)]


resultados = cross_validating_compare(X, y, pipe, ['xgb'], k=10, repeats=1, random_st=1)
resultados

,MAE TRAIN,MAE TRAIN STD,MAE TEST,MAE TEST STD
xgb,1170.66,5.44,1283.51,52.85


In [37]:
xgb_tun = XGBRegressor(eta = 0.07,
                      n_estimators = 300,
                      max_depth = 5,
                      min_child_weight = 1,
                      objective = 'reg:squarederror',
                      subsample = 0.5,
                      base_score = 1,
                      colsample_bynode = 0.55,
                      num_parallel_tree = 1)


pipe1 = Pipeline(steps=[("preprocessor", preprocessor),('modelo', xgb_tun)])

pipe = [TransformedTargetRegressor(regressor=pipe1,
                                   func=np.log, 
                                   inverse_func=np.exp)]


resultados = cross_validating_compare(X, y, pipe, ['xgb'], k=10, repeats=1, random_st=1)
resultados

,MAE TRAIN,MAE TRAIN STD,MAE TEST,MAE TEST STD
xgb,1170.66,5.44,1283.51,52.85


# Avaliando o XGBREgressor com o modo dart como booster


In [31]:
modelo_em_teste = XGBRegressor(objective = 'reg:squarederror')

parametros = {
                'regressor__modelo__booster': ['dart'],
                'regressor__modelo__sample_type': ['uniform', 'weighted'],
                'regressor__modelo__normalize_type': ['tree', 'forest'],
                'regressor__modelo__rate_drop': np.arange(0,1.1,0.1),
                'regressor__modelo__skip_drop': np.arange(0,1.1,0.1)
           
                }


pipe1 = Pipeline(steps=[("preprocessor", preprocessor),('modelo', modelo_em_teste)])

pipe = TransformedTargetRegressor(regressor=pipe1,
                                   func=np.log, 
                                   inverse_func=np.exp)

validation = RepeatedKFold(n_splits=8, n_repeats = 1, random_state = 1)

grid_search_pipe = RandomizedSearchCV(pipe, parametros, cv=validation,
                                      scoring='neg_mean_absolute_error', verbose=2,
                                      return_train_score = True,
                                      n_iter=10,
                                      #n_jobs=-1
                                     )

grid_search_pipe.fit(X, y)

resultados = pd.DataFrame(grid_search_pipe.cv_results_)
resultados.sort_values(by = 'rank_test_score')

Fitting 8 folds for each of 10 candidates, totalling 80 fits
[CV] END regressor__modelo__booster=dart, regressor__modelo__normalize_type=forest, regressor__modelo__rate_drop=0.30000000000000004, regressor__modelo__sample_type=weighted, regressor__modelo__skip_drop=0.1; total time=  12.0s
[CV] END regressor__modelo__booster=dart, regressor__modelo__normalize_type=forest, regressor__modelo__rate_drop=0.30000000000000004, regressor__modelo__sample_type=weighted, regressor__modelo__skip_drop=0.1; total time=  10.6s
[CV] END regressor__modelo__booster=dart, regressor__modelo__normalize_type=forest, regressor__modelo__rate_drop=0.30000000000000004, regressor__modelo__sample_type=weighted, regressor__modelo__skip_drop=0.1; total time=   9.9s
[CV] END regressor__modelo__booster=dart, regressor__modelo__normalize_type=forest, regressor__modelo__rate_drop=0.30000000000000004, regressor__modelo__sample_type=weighted, regressor__modelo__skip_drop=0.1; total time=   7.7s
[CV] END regressor__modelo_

[CV] END regressor__modelo__booster=dart, regressor__modelo__normalize_type=forest, regressor__modelo__rate_drop=0.6000000000000001, regressor__modelo__sample_type=uniform, regressor__modelo__skip_drop=0.4; total time=   6.9s
[CV] END regressor__modelo__booster=dart, regressor__modelo__normalize_type=forest, regressor__modelo__rate_drop=0.6000000000000001, regressor__modelo__sample_type=uniform, regressor__modelo__skip_drop=0.4; total time=   6.9s
[CV] END regressor__modelo__booster=dart, regressor__modelo__normalize_type=tree, regressor__modelo__rate_drop=0.6000000000000001, regressor__modelo__sample_type=weighted, regressor__modelo__skip_drop=0.2; total time=   6.9s
[CV] END regressor__modelo__booster=dart, regressor__modelo__normalize_type=tree, regressor__modelo__rate_drop=0.6000000000000001, regressor__modelo__sample_type=weighted, regressor__modelo__skip_drop=0.2; total time=   6.7s
[CV] END regressor__modelo__booster=dart, regressor__modelo__normalize_type=tree, regressor__model

[CV] END regressor__modelo__booster=dart, regressor__modelo__normalize_type=tree, regressor__modelo__rate_drop=0.0, regressor__modelo__sample_type=weighted, regressor__modelo__skip_drop=0.0; total time=  25.6s
[CV] END regressor__modelo__booster=dart, regressor__modelo__normalize_type=tree, regressor__modelo__rate_drop=0.0, regressor__modelo__sample_type=weighted, regressor__modelo__skip_drop=0.0; total time=  27.2s
[CV] END regressor__modelo__booster=dart, regressor__modelo__normalize_type=tree, regressor__modelo__rate_drop=0.0, regressor__modelo__sample_type=weighted, regressor__modelo__skip_drop=0.0; total time=  25.8s
[CV] END regressor__modelo__booster=dart, regressor__modelo__normalize_type=tree, regressor__modelo__rate_drop=0.0, regressor__modelo__sample_type=weighted, regressor__modelo__skip_drop=0.0; total time=  44.0s
[CV] END regressor__modelo__booster=dart, regressor__modelo__normalize_type=tree, regressor__modelo__rate_drop=0.0, regressor__modelo__sample_type=weighted, reg

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__modelo__skip_drop,param_regressor__modelo__sample_type,param_regressor__modelo__rate_drop,param_regressor__modelo__normalize_type,param_regressor__modelo__booster,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,...,split6_test_score,split7_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,split5_train_score,split6_train_score,split7_train_score,mean_train_score,std_train_score
2,11.359033,0.512686,0.166079,0.024714,0.1,uniform,0.1,tree,dart,"{'regressor__modelo__skip_drop': 0.1, 'regress...",-1398.368881,-1213.637046,-1246.607720,-1357.117383,-1294.779813,...,-1267.957042,-1275.004213,-1290.165738,56.109930,1,-1158.868542,-1186.420387,-1175.221978,-1166.766512,-1167.306845,-1171.932064,-1175.543230,-1177.280875,-1172.417554,7.739759
1,9.734463,1.063113,0.148122,0.021689,0.5,weighted,0.2,forest,dart,"{'regressor__modelo__skip_drop': 0.5, 'regress...",-1411.570279,-1212.269187,-1253.207913,-1356.425021,-1300.149624,...,-1271.842538,-1281.731963,-1293.680011,58.896627,2,-1188.784210,-1212.182082,-1207.286632,-1198.656975,-1202.623207,-1207.466284,-1206.889270,-1209.003612,-1204.111534,6.937324
9,28.846058,5.658726,0.446310,0.181776,0.0,weighted,0.0,tree,dart,"{'regressor__modelo__skip_drop': 0.0, 'regress...",-1394.932946,-1228.033934,-1251.608585,-1368.723959,-1309.321035,...,-1266.027005,-1281.256992,-1296.613316,54.182063,3,-990.092795,-1008.871462,-1001.643533,-998.223481,-1001.436619,-1015.439861,-1007.818736,-999.747108,-1002.909199,7.216777
7,15.505535,5.001234,0.217294,0.035530,0.6,weighted,0.5,tree,dart,{'regressor__modelo__skip_drop': 0.60000000000...,-1390.712766,-1232.145479,-1253.334539,-1374.283528,-1317.036678,...,-1253.433450,-1278.739277,-1297.408393,54.546593,4,-1067.271585,-1106.696651,-1100.082577,-1074.662444,-1091.356598,-1086.417534,-1093.374714,-1096.989167,-1089.606409,12.277674
6,11.147948,2.206708,0.223324,0.067430,0.3,weighted,0.5,tree,dart,{'regressor__modelo__skip_drop': 0.30000000000...,-1399.652519,-1230.197050,-1247.365605,-1362.746176,-1308.607385,...,-1274.447561,-1287.696517,-1297.441920,53.977556,5,-1181.050560,-1215.076666,-1205.134889,-1185.440458,-1198.328631,-1207.097687,-1204.781347,-1200.302511,-1199.651594,10.605935
8,18.873966,8.339461,0.485328,0.187151,0.2,uniform,0.7,tree,dart,"{'regressor__modelo__skip_drop': 0.2, 'regress...",-1411.893086,-1231.881278,-1246.771992,-1373.611926,-1310.673795,...,-1272.934615,-1298.583395,-1301.667178,58.446804,6,-1201.342510,-1225.237894,-1221.293355,-1211.896173,-1216.090495,-1226.482612,-1218.639861,-1227.829306,-1218.601526,8.274153
5,7.891306,1.575122,0.197118,0.115059,0.2,weighted,0.6,tree,dart,"{'regressor__modelo__skip_drop': 0.2, 'regress...",-1421.065978,-1219.797019,-1255.489173,-1375.895565,-1306.363219,...,-1281.235535,-1291.514799,-1303.087341,61.165833,7,-1212.693928,-1235.956241,-1233.527278,-1227.202966,-1227.089487,-1231.646794,-1237.692897,-1227.902273,-1229.213983,7.289235
4,6.944858,0.322576,0.129072,0.015113,0.4,uniform,0.6,forest,dart,"{'regressor__modelo__skip_drop': 0.4, 'regress...",-1446.636822,-1236.204736,-1278.889435,-1393.836185,-1319.856497,...,-1309.297625,-1304.017718,-1320.900444,63.530598,8,-1260.405767,-1284.065593,-1276.052188,-1268.501457,-1274.818469,-1277.147198,-1277.242457,-1280.720218,-1274.869168,6.899010
3,9.840556,1.158546,0.161706,0.030076,0.4,weighted,0.5,forest,dart,"{'regressor__modelo__skip_drop': 0.4, 'regress...",-1455.105086,-1241.118580,-1297.995841,-1395.160365,-1327.064274,...,-1314.472389,-1312.202068,-1328.593987,62.590345,9,-1268.647270,-1296.732203,-1292.639403,-1277.600127,-1291.156310,-1291.099686,-1289.076050,-1291.438330,-1287.298673,8.725511
0,9.070491,1.472634,0.149326,0.038281,0.1,weighted,0.3,forest,dart,"{'regressor__modelo__skip_drop': 0.1, 'regress...",-15

In [42]:
xgb_dart = XGBRegressor(objective = 'reg:squarederror',
                       booster = 'dart',
                       sample_type = 'uniform',
                       normalize_type = 'tree',
                       rate_drop = 0.1,
                       skip_drop = 0.2
                       )

pipe1 = Pipeline(steps=[("preprocessor", preprocessor),('modelo', xgb_dart)])

pipe = [TransformedTargetRegressor(regressor=pipe1,
                                   func=np.log, 
                                   inverse_func=np.exp)]


resultados = cross_validating_compare(X, y, pipe, ['xgb'], k=10, repeats=1, random_st=1)
resultados

,MAE TRAIN,MAE TRAIN STD,MAE TEST,MAE TEST STD
xgb,1154.77,5.56,1290.41,60.08


## Avaliando o BaggingRegressor

In [43]:


modelo_em_teste = BaggingRegressor()

parametros = {   
                 'regressor__modelo__max_samples' : [0.5, 1.0],
                 'regressor__modelo__n_estimators':  np.arange(25,201,25),
                 'regressor__modelo__bootstrap': [True, False],
                 'regressor__modelo__bootstrap_features': [True, False]
                 }

pipe1 = Pipeline(steps=[("preprocessor", preprocessor),('modelo', modelo_em_teste)])

pipe = TransformedTargetRegressor(regressor=pipe1,
                                   func=np.log, 
                                   inverse_func=np.exp)

validation = RepeatedKFold(n_splits=8, n_repeats = 1, random_state = 1)

grid_search_pipe = RandomizedSearchCV(pipe, parametros, cv=validation,
                                      scoring='neg_mean_absolute_error', verbose=2,
                                      return_train_score = True,
                                      n_iter=20,
                                      #n_jobs=-1
                                     )

grid_search_pipe.fit(X, y)

resultados = pd.DataFrame(grid_search_pipe.cv_results_)
resultados.sort_values(by = 'rank_test_score')

Fitting 8 folds for each of 20 candidates, totalling 160 fits
[CV] END regressor__modelo__bootstrap=True, regressor__modelo__bootstrap_features=True, regressor__modelo__max_samples=0.5, regressor__modelo__n_estimators=175; total time=   8.0s
[CV] END regressor__modelo__bootstrap=True, regressor__modelo__bootstrap_features=True, regressor__modelo__max_samples=0.5, regressor__modelo__n_estimators=175; total time=   8.7s
[CV] END regressor__modelo__bootstrap=True, regressor__modelo__bootstrap_features=True, regressor__modelo__max_samples=0.5, regressor__modelo__n_estimators=175; total time=   8.5s
[CV] END regressor__modelo__bootstrap=True, regressor__modelo__bootstrap_features=True, regressor__modelo__max_samples=0.5, regressor__modelo__n_estimators=175; total time=   6.8s
[CV] END regressor__modelo__bootstrap=True, regressor__modelo__bootstrap_features=True, regressor__modelo__max_samples=0.5, regressor__modelo__n_estimators=175; total time=   7.4s
[CV] END regressor__modelo__bootstrap=

[CV] END regressor__modelo__bootstrap=True, regressor__modelo__bootstrap_features=True, regressor__modelo__max_samples=0.5, regressor__modelo__n_estimators=100; total time=   3.9s
[CV] END regressor__modelo__bootstrap=True, regressor__modelo__bootstrap_features=True, regressor__modelo__max_samples=0.5, regressor__modelo__n_estimators=100; total time=   3.7s
[CV] END regressor__modelo__bootstrap=True, regressor__modelo__bootstrap_features=True, regressor__modelo__max_samples=0.5, regressor__modelo__n_estimators=100; total time=   3.3s
[CV] END regressor__modelo__bootstrap=False, regressor__modelo__bootstrap_features=False, regressor__modelo__max_samples=1.0, regressor__modelo__n_estimators=150; total time=  20.9s
[CV] END regressor__modelo__bootstrap=False, regressor__modelo__bootstrap_features=False, regressor__modelo__max_samples=1.0, regressor__modelo__n_estimators=150; total time=  16.3s
[CV] END regressor__modelo__bootstrap=False, regressor__modelo__bootstrap_features=False, regres

[CV] END regressor__modelo__bootstrap=True, regressor__modelo__bootstrap_features=True, regressor__modelo__max_samples=1.0, regressor__modelo__n_estimators=25; total time=   2.3s
[CV] END regressor__modelo__bootstrap=True, regressor__modelo__bootstrap_features=True, regressor__modelo__max_samples=1.0, regressor__modelo__n_estimators=25; total time=   2.0s
[CV] END regressor__modelo__bootstrap=True, regressor__modelo__bootstrap_features=True, regressor__modelo__max_samples=1.0, regressor__modelo__n_estimators=25; total time=   1.7s
[CV] END regressor__modelo__bootstrap=True, regressor__modelo__bootstrap_features=True, regressor__modelo__max_samples=1.0, regressor__modelo__n_estimators=25; total time=   1.5s
[CV] END regressor__modelo__bootstrap=True, regressor__modelo__bootstrap_features=True, regressor__modelo__max_samples=1.0, regressor__modelo__n_estimators=25; total time=   1.6s
[CV] END regressor__modelo__bootstrap=True, regressor__modelo__bootstrap_features=False, regressor__model

[CV] END regressor__modelo__bootstrap=False, regressor__modelo__bootstrap_features=True, regressor__modelo__max_samples=0.5, regressor__modelo__n_estimators=175; total time=   8.6s
[CV] END regressor__modelo__bootstrap=False, regressor__modelo__bootstrap_features=True, regressor__modelo__max_samples=0.5, regressor__modelo__n_estimators=175; total time=   8.3s
[CV] END regressor__modelo__bootstrap=False, regressor__modelo__bootstrap_features=True, regressor__modelo__max_samples=0.5, regressor__modelo__n_estimators=175; total time=   7.8s
[CV] END regressor__modelo__bootstrap=False, regressor__modelo__bootstrap_features=True, regressor__modelo__max_samples=0.5, regressor__modelo__n_estimators=175; total time=   7.6s
[CV] END regressor__modelo__bootstrap=False, regressor__modelo__bootstrap_features=True, regressor__modelo__max_samples=0.5, regressor__modelo__n_estimators=175; total time=   7.4s
[CV] END regressor__modelo__bootstrap=False, regressor__modelo__bootstrap_features=True, regres

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__modelo__n_estimators,param_regressor__modelo__max_samples,param_regressor__modelo__bootstrap_features,param_regressor__modelo__bootstrap,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,split5_train_score,split6_train_score,split7_train_score,mean_train_score,std_train_score
12,8.012697,0.196860,0.226395,0.020572,175,0.5,False,True,"{'regressor__modelo__n_estimators': 175, 'regr...",-1451.206404,-1238.243459,-1289.329365,-1379.471273,-1291.713834,-1270.913991,-1273.605137,-1303.403420,-1312.235860,64.741786,1,-849.314167,-870.069869,-866.692314,-854.511267,-866.592474,-869.932673,-869.620620,-866.188058,-864.115180,7.314301
3,11.112032,0.635089,0.382104,0.135286,200,0.5,False,True,"{'regressor__modelo__n_estimators': 200, 'regr...",-1447.961317,-1241.062616,-1292.881751,-1376.860270,-1292.354433,-1274.649799,-1284.636143,-1309.108172,-1314.939313,61.805104,2,-846.883708,-873.017501,-865.689613,-856.380159,-867.202061,-866.324135,-868.965000,-866.648592,-863.888846,7.765483
15,6.470322,0.793929,0.188621,0.024781,125,0.5,False,True,"{'regressor__modelo__n_estimators': 125, 'regr...",-1449.413723,-1252.333319,-1294.298502,-1379.549429,-1284.702290,-1271.791915,-1273.432878,-1315.918911,-1315.180121,62.352469,3,-853.158718,-872.668923,-863.446275,-855.223220,-863.680987,-870.380420,-870.186878,-865.576965,-864.290298,6.623101
9,3.353407,0.177462,0.108710,0.011383,75,0.5,False,True,"{'regressor__modelo__n_estimators': 75, 'regre...",-1453.570422,-1241.637907,-1294.962997,-1382.867015,-1292.241735,-1281.227309,-1270.118618,-1307.294227,-1315.490029,64.538265,4,-853.361144,-875.403313,-867.119618,-856.134860,-867.776755,-872.507623,-870.958293,-871.423664,-866.835659,7.421740
8,4.487265,0.116402,0.145237,0.014674,100,0.5,False,True,"{'regressor__modelo__n_estimators': 100, 'regr...",-1454.316306,-1252.372654,-1296.062826,-1379.146627,-1291.169457,-1273.131444,-1279.989560,-1310.740507,-1317.116173,62.583488,5,-850.444141,-871.283302,-867.391054,-856.265331,-867.401442,-871.674198,-869.937168,-864.891234,-864.910984,7.141432
16,1.360237,0.103650,0.061212,0.023410,25,0.5,False,True,"{'regressor__modelo__n_estimators': 25, 'regre...",-1476.771600,-1252.332938,-1302.367884,-1381.274576,-1285.850068,-1289.651646,-1287.554680,-1315.780453,-1323.947981,67.318908,6,-867.968174,-886.090644,-889.464840,-871.193505,-878.188081,-882.535411,-880.302294,-877.565891,-879.163605,6.700089
17,8.193589,0.679670,0.199591,0.028531,175,0.5,True,False,"{'regressor__modelo__n_estimators': 175, 'regr...",-1471.608273,-1255.801498,-1298.204176,-1381.145144,-1311.424541,-1279.968939,-1320.569113,-1330.369573,-1331.136407,63.356158,7,-989.690518,-992.624834,-978.402918,-960.047129,-985.188517,-1007.065074,-958.869402,-977.334789,-981.152898,15.233070
1,5.152098,0.581601,0.158326,0.069274,100,0.5,True,False,"{'regressor__modelo__n_estimators': 100, 'regr...",-1461.602863,-1260.869271,-1302.270449,-1411.933078,-1304.911786,-1285.125969,-1313.430236,-1330.806163,-1333.868727,63.618881,8,-870.383923,-979.625805,-967.276344,-1018.689191,-904.105710,-969.310630,-978.299808,-988.170082,-959.482687,45.070410
5,3.865289,0.386835,0.140251,0.020991,100,0.5,True,True,"{'regressor__modelo__n_estimators': 100, 'regr...",-1474.886372,-1273.222689,-1296.398447,-1380.350343,-1328.883861,-1282.581022,-1322.729490,-1333.167944,-1336.527521,61.085778,9,-1050.485174,-1124.332669,-1054.723115,-1032.239703,-1059.269887,-1043.310374,-1051.251329,-1055.586381,-1058.899829,25.964198
0,7.523256,0.623833,0.225772,0.029921,175,0.5,True,True,"{'regressor__modelo__n_estimators': 175, 'regr...",-1462.389453,-1270.268546,-1308.546152,-1401.097894,-1313.123804,-1281.945788,-1323.3

## Avaliando o Gradient Boosting Regressor

In [44]:
modelo_em_teste = GradientBoostingRegressor(loss = 'lad' ) # lad => mean absolute error

parametros = {
                  
                   'regressor__modelo__min_samples_split': np.arange(2,12),
                   'regressor__modelo__min_samples_leaf': np.arange(1,11),
                   'regressor__modelo__max_depth' : np.arange(2,15),
                   'regressor__modelo__max_features': ['sqrt', 'auto', 'log2'],
                   'regressor__modelo__n_estimators': np.arange(50,1000,25),
                   'regressor__modelo__learning_rate': np.arange(0.01, 0.4, 0.02)
                  },   


pipe1 = Pipeline(steps=[("preprocessor", preprocessor),('modelo', modelo_em_teste)])

pipe = TransformedTargetRegressor(regressor=pipe1,
                                   func=np.log, 
                                   inverse_func=np.exp)

validation = RepeatedKFold(n_splits=8, n_repeats = 1, random_state = 1)

grid_search_pipe = RandomizedSearchCV(pipe, parametros, cv=validation,
                                      scoring='neg_mean_absolute_error', verbose=2,
                                      return_train_score = True,
                                      n_iter=20,
                                      #n_jobs=-1
                                     )

grid_search_pipe.fit(X, y)

resultados = pd.DataFrame(grid_search_pipe.cv_results_)
resultados.sort_values(by = 'rank_test_score')



Fitting 8 folds for each of 20 candidates, totalling 160 fits
[CV] END regressor__modelo__learning_rate=0.12999999999999998, regressor__modelo__max_depth=5, regressor__modelo__max_features=sqrt, regressor__modelo__min_samples_leaf=4, regressor__modelo__min_samples_split=4, regressor__modelo__n_estimators=725; total time=  22.7s
[CV] END regressor__modelo__learning_rate=0.12999999999999998, regressor__modelo__max_depth=5, regressor__modelo__max_features=sqrt, regressor__modelo__min_samples_leaf=4, regressor__modelo__min_samples_split=4, regressor__modelo__n_estimators=725; total time=  19.4s
[CV] END regressor__modelo__learning_rate=0.12999999999999998, regressor__modelo__max_depth=5, regressor__modelo__max_features=sqrt, regressor__modelo__min_samples_leaf=4, regressor__modelo__min_samples_split=4, regressor__modelo__n_estimators=725; total time=  19.6s
[CV] END regressor__modelo__learning_rate=0.12999999999999998, regressor__modelo__max_depth=5, regressor__modelo__max_features=sqrt, r

[CV] END regressor__modelo__learning_rate=0.30999999999999994, regressor__modelo__max_depth=11, regressor__modelo__max_features=log2, regressor__modelo__min_samples_leaf=4, regressor__modelo__min_samples_split=7, regressor__modelo__n_estimators=800; total time= 1.6min
[CV] END regressor__modelo__learning_rate=0.35, regressor__modelo__max_depth=6, regressor__modelo__max_features=auto, regressor__modelo__min_samples_leaf=7, regressor__modelo__min_samples_split=11, regressor__modelo__n_estimators=550; total time=  26.6s
[CV] END regressor__modelo__learning_rate=0.35, regressor__modelo__max_depth=6, regressor__modelo__max_features=auto, regressor__modelo__min_samples_leaf=7, regressor__modelo__min_samples_split=11, regressor__modelo__n_estimators=550; total time=  25.7s
[CV] END regressor__modelo__learning_rate=0.35, regressor__modelo__max_depth=6, regressor__modelo__max_features=auto, regressor__modelo__min_samples_leaf=7, regressor__modelo__min_samples_split=11, regressor__modelo__n_esti

[CV] END regressor__modelo__learning_rate=0.01, regressor__modelo__max_depth=9, regressor__modelo__max_features=auto, regressor__modelo__min_samples_leaf=3, regressor__modelo__min_samples_split=8, regressor__modelo__n_estimators=625; total time= 1.5min
[CV] END regressor__modelo__learning_rate=0.03, regressor__modelo__max_depth=2, regressor__modelo__max_features=sqrt, regressor__modelo__min_samples_leaf=8, regressor__modelo__min_samples_split=4, regressor__modelo__n_estimators=625; total time=   6.7s
[CV] END regressor__modelo__learning_rate=0.03, regressor__modelo__max_depth=2, regressor__modelo__max_features=sqrt, regressor__modelo__min_samples_leaf=8, regressor__modelo__min_samples_split=4, regressor__modelo__n_estimators=625; total time=   7.3s
[CV] END regressor__modelo__learning_rate=0.03, regressor__modelo__max_depth=2, regressor__modelo__max_features=sqrt, regressor__modelo__min_samples_leaf=8, regressor__modelo__min_samples_split=4, regressor__modelo__n_estimators=625; total t

[CV] END regressor__modelo__learning_rate=0.26999999999999996, regressor__modelo__max_depth=7, regressor__modelo__max_features=sqrt, regressor__modelo__min_samples_leaf=5, regressor__modelo__min_samples_split=2, regressor__modelo__n_estimators=925; total time=  40.2s
[CV] END regressor__modelo__learning_rate=0.24999999999999997, regressor__modelo__max_depth=11, regressor__modelo__max_features=sqrt, regressor__modelo__min_samples_leaf=10, regressor__modelo__min_samples_split=10, regressor__modelo__n_estimators=725; total time=  58.7s
[CV] END regressor__modelo__learning_rate=0.24999999999999997, regressor__modelo__max_depth=11, regressor__modelo__max_features=sqrt, regressor__modelo__min_samples_leaf=10, regressor__modelo__min_samples_split=10, regressor__modelo__n_estimators=725; total time=  59.1s
[CV] END regressor__modelo__learning_rate=0.24999999999999997, regressor__modelo__max_depth=11, regressor__modelo__max_features=sqrt, regressor__modelo__min_samples_leaf=10, regressor__model

[CV] END regressor__modelo__learning_rate=0.12999999999999998, regressor__modelo__max_depth=12, regressor__modelo__max_features=auto, regressor__modelo__min_samples_leaf=3, regressor__modelo__min_samples_split=9, regressor__modelo__n_estimators=250; total time=  41.5s
[CV] END regressor__modelo__learning_rate=0.12999999999999998, regressor__modelo__max_depth=12, regressor__modelo__max_features=auto, regressor__modelo__min_samples_leaf=3, regressor__modelo__min_samples_split=9, regressor__modelo__n_estimators=250; total time=  33.2s
[CV] END regressor__modelo__learning_rate=0.36999999999999994, regressor__modelo__max_depth=5, regressor__modelo__max_features=sqrt, regressor__modelo__min_samples_leaf=1, regressor__modelo__min_samples_split=9, regressor__modelo__n_estimators=725; total time=  15.8s
[CV] END regressor__modelo__learning_rate=0.36999999999999994, regressor__modelo__max_depth=5, regressor__modelo__max_features=sqrt, regressor__modelo__min_samples_leaf=1, regressor__modelo__min

[CV] END regressor__modelo__learning_rate=0.03, regressor__modelo__max_depth=13, regressor__modelo__max_features=auto, regressor__modelo__min_samples_leaf=6, regressor__modelo__min_samples_split=5, regressor__modelo__n_estimators=750; total time= 2.2min
[CV] END regressor__modelo__learning_rate=0.03, regressor__modelo__max_depth=13, regressor__modelo__max_features=auto, regressor__modelo__min_samples_leaf=6, regressor__modelo__min_samples_split=5, regressor__modelo__n_estimators=750; total time= 2.3min
[CV] END regressor__modelo__learning_rate=0.03, regressor__modelo__max_depth=13, regressor__modelo__max_features=auto, regressor__modelo__min_samples_leaf=6, regressor__modelo__min_samples_split=5, regressor__modelo__n_estimators=750; total time= 2.6min


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__modelo__n_estimators,param_regressor__modelo__min_samples_split,param_regressor__modelo__min_samples_leaf,param_regressor__modelo__max_features,param_regressor__modelo__max_depth,param_regressor__modelo__learning_rate,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,split6_test_score,split7_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,split5_train_score,split6_train_score,split7_train_score,mean_train_score,std_train_score
2,51.320271,3.814285,0.145361,0.081761,575,9,2,sqrt,10,0.11,"{'regressor__modelo__n_estimators': 575, 'regr...",-1386.195829,-1225.102608,-1230.766992,-1345.163251,...,-1263.804819,-1264.298115,-1280.354141,52.620338,1,-954.959096,-975.021336,-986.372535,-941.286622,-980.543466,-976.543336,-977.824677,-978.249098,-971.350021,14.197857
0,20.262270,1.551836,0.084236,0.006324,725,4,4,sqrt,5,0.13,"{'regressor__modelo__n_estimators': 725, 'regr...",-1398.494601,-1209.852705,-1243.081799,-1355.129737,...,-1264.080281,-1272.381983,-1286.523180,58.340318,2,-1138.484583,-1158.888647,-1148.100230,-1129.857613,-1151.624783,-1146.559014,-1141.262707,-1151.753185,-1145.816345,8.476499
7,95.596972,7.391329,0.149826,0.012779,625,8,3,auto,9,0.01,"{'regressor__modelo__n_estimators': 625, 'regr...",-1404.624633,-1219.911091,-1254.726418,-1345.628129,...,-1266.562472,-1274.087046,-1287.402926,55.743612,3,-1084.479078,-1108.700364,-1100.346885,-1088.233652,-1091.436386,-1092.775398,-1099.778097,-1105.035709,-1096.348196,7.901070
19,148.365239,9.802179,0.203207,0.038111,750,5,6,auto,13,0.03,"{'regressor__modelo__n_estimators': 750, 'regr...",-1402.743185,-1238.372313,-1257.649862,-1345.702657,...,-1271.095083,-1279.634424,-1292.066840,52.216269,4,-886.280866,-884.075324,-901.243686,-876.967831,-892.759748,-876.744043,-897.378507,-908.741689,-890.523962,10.793663
16,16.634640,1.125660,0.072805,0.009670,725,9,1,sqrt,5,0.37,"{'regressor__modelo__n_estimators': 725, 'regr...",-1405.195798,-1210.641249,-1252.138618,-1343.672897,...,-1296.096138,-1281.868021,-1294.198434,55.997168,5,-1120.675077,-1124.563221,-1132.348470,-1116.233432,-1119.597292,-1128.759772,-1136.375620,-1115.188913,-1124.217724,7.192358
17,17.480629,0.837835,0.064950,0.013075,725,5,8,auto,3,0.11,"{'regressor__modelo__n_estimators': 725, 'regr...",-1412.948183,-1224.520725,-1257.003850,-1358.389038,...,-1266.042167,-1284.631086,-1294.730296,57.496800,6,-1201.213563,-1225.133064,-1234.467865,-1217.521527,-1224.218814,-1235.499539,-1223.512672,-1227.111110,-1223.584769,10.080159
1,124.407639,7.742888,0.206074,0.028511,700,11,6,sqrt,14,0.09,"{'regressor__modelo__n_estimators': 700, 'regr...",-1405.590742,-1234.611955,-1273.617222,-1359.559318,...,-1269.777455,-1271.086526,-1297.480190,52.707117,7,-795.624393,-798.071871,-795.445683,-772.545236,-793.225993,-779.086870,-784.659144,-796.892253,-789.443931,8.903446
14,10.975276,0.713695,0.043010,0.004794,50,11,1,auto,13,0.19,"{'regressor__modelo__n_estimators': 50, 'regre...",-1418.636115,-1248.158889,-1266.917626,-1349.838196,...,-1265.845734,-1274.582946,-1297.489605,55.270402,8,-941.992070,-941.441700,-955.391394,-930.187036,-941.101920,-947.954346,-965.929213,-945.819489,-946.227146,9.998086
15,36.273562,2.601247,0.071932,0.011900,250,9,3,auto,12,0.13,"{'regressor__modelo__n_estimators': 250, 'regr...",-1409.899033,-1255.513562,-1258.780387,-1347.248483,...,-1259.974824,-1275.493613,-1297.817937,53.157546,9,-871.166862,-880.348195,-877.184213,-888.090129,-877.737386,-877.719211,-890.812900,-895.420307,-882.309901,7.696309
10,58.648661,2.720003,0.160446,0.042724,600,7,6,log2,11,0.19,"{'regressor__modelo__n_estimators': 600, 'regr...",-1409.978509,-1248.875612,-1255.009185,-1369.624896,...,-1270.227213,-1273.493002,-1298.513673,56.799503,10,-868.433325,-878.944425,-866.744390,-861.402617,-866.365403,-870.68

In [46]:
grid_search_pipe.best_params_

{'regressor__modelo__n_estimators': 575,
 'regressor__modelo__min_samples_split': 9,
 'regressor__modelo__min_samples_leaf': 2,
 'regressor__modelo__max_features': 'sqrt',
 'regressor__modelo__max_depth': 10,
 'regressor__modelo__learning_rate': 0.10999999999999997}

In [60]:
grad_tun = GradientBoostingRegressor(loss = 'lad', 
                                    n_estimators = 575,
                                    min_samples_split = 9,
                                    min_samples_leaf = 2,
                                    max_features = 'sqrt',
                                    max_depth = 10,
                                    learning_rate = 0.10999999999999997,
                                    
                                    validation_fraction=0.2, 
                                    n_iter_no_change = 1,
                                    subsample = 1,
                                    tol = 0.00001
                                    )

pipe1 = Pipeline(steps=[("preprocessor", preprocessor),('modelo', grad_tun)])

pipe = [TransformedTargetRegressor(regressor=pipe1,
                                   func=np.log, 
                                   inverse_func=np.exp)]


resultados = cross_validating_compare(X, y, pipe, ['xgb'], k=10, repeats=1, random_st=1)
resultados

,MAE TRAIN,MAE TRAIN STD,MAE TEST,MAE TEST STD
xgb,1143.42,11.11,1290.86,59.92


# Avaliando o MLPRegressor - Redes Neurais


In [68]:
modelo_em_teste = MLPRegressor(hidden_layer_sizes = (300,300))

parametros = {
                   'regressor__modelo__activation': ['identity', 'logistic', 'tanh', 'relu'],
                   'regressor__modelo__solver': ['lbfgs', 'sgd', 'adam'],
                   'regressor__modelo__validation_fraction': np.arange(0.1, 1.0, 0.1),
                   'regressor__modelo__learning_rate_init': np.arange(0.01, 0.5, 0.1),
                   'regressor__modelo__learning_rate': ['constant', 'invscaling', 'adaptive'],
                   'regressor__modelo__max_iter': np.arange(100, 2000, 50)                               
                    }

pipe1 = Pipeline(steps=[("preprocessor", preprocessor),('modelo', modelo_em_teste)])

pipe = TransformedTargetRegressor(regressor=pipe1,
                                   func=np.log, 
                                   inverse_func=np.exp)

validation = RepeatedKFold(n_splits=8, n_repeats = 1, random_state = 1)

grid_search_pipe = RandomizedSearchCV(pipe, parametros, cv=validation,
                                      scoring='neg_mean_absolute_error', verbose=2,
                                      return_train_score = True,
                                      n_iter=50,
                                      #n_jobs=-1
                                     )

grid_search_pipe.fit(X, y)

resultados = pd.DataFrame(grid_search_pipe.cv_results_)
resultados.sort_values(by = 'rank_test_score')



Fitting 8 folds for each of 50 candidates, totalling 400 fits
[CV] END regressor__modelo__activation=identity, regressor__modelo__learning_rate=constant, regressor__modelo__learning_rate_init=0.11, regressor__modelo__max_iter=1750, regressor__modelo__solver=lbfgs, regressor__modelo__validation_fraction=0.2; total time=  22.8s
[CV] END regressor__modelo__activation=identity, regressor__modelo__learning_rate=constant, regressor__modelo__learning_rate_init=0.11, regressor__modelo__max_iter=1750, regressor__modelo__solver=lbfgs, regressor__modelo__validation_fraction=0.2; total time=  27.2s
[CV] END regressor__modelo__activation=identity, regressor__modelo__learning_rate=constant, regressor__modelo__learning_rate_init=0.11, regressor__modelo__max_iter=1750, regressor__modelo__solver=lbfgs, regressor__modelo__validation_fraction=0.2; total time=  24.6s
[CV] END regressor__modelo__activation=identity, regressor__modelo__learning_rate=constant, regressor__modelo__learning_rate_init=0.11, regr

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\extmath.py:152: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\extmath.py:152: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (550) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __

[CV] END regressor__modelo__activation=identity, regressor__modelo__learning_rate=adaptive, regressor__modelo__learning_rate_init=0.41000000000000003, regressor__modelo__max_iter=550, regressor__modelo__solver=sgd, regressor__modelo__validation_fraction=0.6; total time= 2.6min


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\extmath.py:152: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\extmath.py:152: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (550) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __

[CV] END regressor__modelo__activation=identity, regressor__modelo__learning_rate=adaptive, regressor__modelo__learning_rate_init=0.41000000000000003, regressor__modelo__max_iter=550, regressor__modelo__solver=sgd, regressor__modelo__validation_fraction=0.6; total time= 2.6min


...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[CV] END regressor__modelo__activation=relu, regressor__modelo__learning_rate=constant, regressor__modelo__learning_rate_init=0.41000000000000003, regressor__modelo__max_iter=450, regressor__modelo__solver=lbfgs, regressor__modelo__validation_fraction=0.8; total time= 2.7min


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[CV] END regressor__modelo__activation=relu, regressor__modelo__learning_rate=constant, regressor__modelo__learning_rate_init=0.41000000000000003, regressor__modelo__max_iter=450, regressor__modelo__solver=lbfgs, regressor__modelo__validation_fraction=0.8; total time= 2.8min


KeyboardInterrupt: 

In [63]:
grid_search_pipe.best_params_

{'regressor__modelo__validation_fraction': 0.6,
 'regressor__modelo__solver': 'lbfgs',
 'regressor__modelo__max_iter': 1000,
 'regressor__modelo__learning_rate_init': 0.21000000000000002,
 'regressor__modelo__learning_rate': 'adaptive',
 'regressor__modelo__activation': 'relu'}

In [ ]:
# ema 1325
rede_tun = MLPRegressor(hidden_layer_sizes = (300,300),
                       validation_fraction = 0.6,
                       solver = 'lbfgs',
                       max_iter = 1000,
                       learning_rate_init = 0.21000000000000002,
                       learning_rate = 'adaptive',
                       activation = 'relu')

# Agregando os modelos através do VotingRegressor

In [9]:
# 1283
outro_xgb = XGBRegressor(learning_rate = 0.01, 
                        n_estimators = 2200, 
                        subsample = 0.9000000000000004, 
                        colsample_bynode = 0.7000000000000002, 
                        objective = 'reg:squarederror',
                        base_score = 0.3,
                        num_parallel_tree = 1,
                        min_child_weight = 2,
                        max_depth = 4
                       )

# 1283
xgb_tun = XGBRegressor(eta = 0.07,
                      n_estimators = 300,
                      max_depth = 5,
                      min_child_weight = 1,
                      objective = 'reg:squarederror',
                      subsample = 0.5,
                      base_score = 1,
                      colsample_bynode = 0.55,
                      num_parallel_tree = 1)

# 1290
xgb_dart = XGBRegressor(objective = 'reg:squarederror',
                       booster = 'dart',
                       sample_type = 'uniform',
                       normalize_type = 'tree',
                       rate_drop = 0.1,
                       skip_drop = 0.2
                       )

rede_tun = MLPRegressor(hidden_layer_sizes = (300,300),
                       validation_fraction = 0.6,
                       solver = 'lbfgs',
                       max_iter = 1000,
                       learning_rate_init = 0.21000000000000002,
                       learning_rate = 'adaptive',
                       activation = 'relu')



xgb_sub2 = XGBRegressor(learning_rate = 0.06, 
                        n_estimators = 200, 
                        subsample = 0.85, 
                        colsample_bynode = 0.75, 
                        objective = 'reg:squarederror',
                        eval_metric = 'mae',
                       )


gbr_lad2 = GradientBoostingRegressor(n_estimators = 150, 
                                     min_samples_split = 7, 
                                     min_samples_leaf = 4, 
                                     max_features = 'log2', max_depth = 13, 
                                     loss = 'lad', learning_rate = 0.09,
                                     validation_fraction=0.2, 
                                     subsample = 0.75
                                    )

modelos_xgb1 = XGBRegressor(subsample = 0.8000000000000003, 
                           num_parallel_tree = 2, 
                           n_estimators = 950, 
                           max_depth = 6, 
                           eta = 0.02, 
                           colsample_bynode = 1.0000000000000004, 
                           booster = 'gbtree', 
                           base_score = 0.8499999999999999, 
                           eval_metric = 'mae',
                           )

modelos_xgb = XGBRegressor(subsample = 0.8000000000000003, 
                           num_parallel_tree = 2, 
                           n_estimators = 950, 
                           max_depth = 5, 
                           eta = 0.02, 
                           colsample_bynode = 1.0000000000000004, 
                           booster = 'gbtree', 
                           base_score = 0.8499999999999999, 
                           eval_metric = 'mae',
                           )

In [69]:


X = df_train.drop(columns = ['remuneracao', 'id', 'idade'], axis = 1)
y =df_train['remuneracao']

er = VotingRegressor([('xgb sub2', xgb_sub2), 
                      ('outro_xgb', outro_xgb),
                      ('xgb_tun', xgb_tun), 
                      ('xgb modelo', modelos_xgb), 
                      ('xgb dart', xgb_dart),                 
                      ('redes neurais', rede_tun), 
                      ('gbr2', gbr_lad2),
                      ('modelo_2', modelos_xgb1)
                     ])

pipe1 = Pipeline(steps=[("preprocessor", preprocessor),('voting', er)])

pipe = [TransformedTargetRegressor(regressor=pipe1,
                                   func=np.log, 
                                   inverse_func=np.exp)]

resultados = cross_validating_compare(X, y, pipe, ['Voting'], k=10, repeats=1, random_st=1)
resultados

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITE

,MAE TRAIN,MAE TRAIN STD,MAE TEST,MAE TEST STD
Voting,1130.07,5.59,1275.19,58.44


# MODELO FINAL

In [10]:

tempo_pipeline = Pipeline(
    steps = [
        ("trans1", no_change),
        ("scaler", log1),
        ])

horas_pipeline = Pipeline(
    steps = [
        ('zeradas', horas_zero),
        ("trans2", horas_trab1),
        ("scaler", log1),
        ])

cor_pipeline = Pipeline(
    steps = [
        ("trans3", no_change),
        ("scaler", min_max),
        ])

escolar_pipeline = Pipeline(
    steps = [
        ("trans4", no_change),
        ("scaler", no_change),
        ])

empresa_pipeline = Pipeline(
    steps = [
        ("trans5", empresa1),
        ("scaler", robust),
        ])

f_etaria_pipeline = Pipeline(
    steps = [
        ("trans6", f_eta),
        ("scaler", min_max),
        ])

preprocessor = ColumnTransformer([('tempo', tempo_pipeline, ['tempo_no_emprego']), 
                                 ('horas_trabalho', horas_pipeline, ['horas_trabalho_por_semana']), 
                                 ('cor', cor_pipeline , ['cor_raca']),
                                 ('escolar', escolar_pipeline,['escolaridade']),
                                 ('empresa', empresa_pipeline, ['tamanho_da_empresa']),
                                 ('f_etaria', f_etaria_pipeline, ['faixa_etaria']),
                                 ('sex', no_change, ['sexo'])
                          
                                 ],
                                remainder='passthrough')

In [34]:
er = VotingRegressor([('xgb sub2', xgb_sub2), 
                      ('outro_xgb', outro_xgb),
                      ('xgb_tun', xgb_tun), 
                      ('xgb modelo', modelos_xgb), 
                      ('xgb dart', xgb_dart),                 
                      ('grad boot_lad', modelo_base), 
                      ('gbr2', gbr_lad2),
                      ('modelo_2', modelos_xgb1)
                     ])


pipe1 = Pipeline(steps=[("preprocessor", preprocessor),('voting', er)])

modelo_final_1 = TransformedTargetRegressor(regressor=pipe1,
                                   func=np.log, 
                                   inverse_func=np.exp)



In [75]:
df_treinando = df_train.copy()
df_testando = df_test.copy()


X_final = df_treinando.drop(columns = ['remuneracao', 'id', 'idade'], axis = 1)
y_final = df_treinando['remuneracao']


modelo_final_1.fit(X_final, y_final)

df_testando_ = df_testando.drop(columns = ['id', 'idade'], axis = 1)
preditos_final = modelo_final_1.predict(df_testando_)


df_testando['preditos'] = preditos_final 

# ALTERANDO OS VALORES DOS DADOS DUPLICADOS

In [92]:
# Duplicated Indexes: changing the ones that I know the real wage

for indice, linha in df_treinando.iterrows():
    if linha.id in indices_duplicados:
        ident = int(linha.id)
        df_testando.loc[df_testando.id == ident, 'preditos'] = linha.remuneracao

# SUBMISSÃO 

#### Nessa submissão, o modelo estava com um EMA por volta de 630

In [93]:
subm = pd.DataFrame()
subm['remuneracao'] = df_testando.preditos

In [95]:
subm.to_csv('submissão.csv', index = False)

# <font color=green size=10> Deploy do Modelo

In [1]:
!pip install joblib
import joblib

In [36]:
joblib.dump(modelo_final_1, 'modelo_wage_7flai.joblib')

['modelo_wage_7flai.joblib']

# <font color=red> Análise da clusterização
    
A análise visual permitiu notar que havia alguns valores que podiam ser encarados pelo modelo como outliers e assim, tendo seu valor sub ou superestimado.
    
Dessa forma, optou-se por uma análise de uma possível clusterização (Nota alunos FLAI: conforme dica da prof Juliana)

In [14]:
df_cor_n99 = df_train[df_train.cor_raca != 99]

X1 = df_cor_n99.drop(columns = ['remuneracao', 'id', 'idade'], axis = 1)
y1 =df_cor_n99['remuneracao']

gbr_lad2 = GradientBoostingRegressor(n_estimators = 150, 
                                     min_samples_split = 7, 
                                     min_samples_leaf = 4, 
                                     max_features = 'log2', max_depth = 13, 
                                     loss = 'lad', learning_rate = 0.09,
                                     validation_fraction=0.2, 
                                     subsample = 0.75
                                    )





pipe1 = Pipeline(steps=[("preprocessor", preprocessor),('gbr', gbr_lad2)])

pipe = [TransformedTargetRegressor(regressor=pipe1,
                                   func=np.log, 
                                   inverse_func=np.exp)]

resultados = cross_validating_compare(X1, y1, pipe, ['Voting'], k=10, repeats=1, random_st=1)
resultados

,MAE TRAIN,MAE TRAIN STD,MAE TEST,MAE TEST STD
Voting,532.14,3.94,839.38,48.27


In [15]:
df_empresa = df_train[df_train.tamanho_da_empresa != 10]

X1 = df_empresa.drop(columns = ['remuneracao', 'id', 'idade'], axis = 1)
y1 =df_empresa['remuneracao']

resultados = cross_validating_compare(X1, y1, pipe, ['Voting'], k=10, repeats=1, random_st=1)
resultados

,MAE TRAIN,MAE TRAIN STD,MAE TEST,MAE TEST STD
Voting,633.92,9.5,990.06,76.44


In [16]:
top_tempo_emprego = df_train['tempo_no_emprego'].quantile(0.975)
media_tempo_emprego = df_train['tempo_no_emprego'].mean()

df_tempo_emprego = df_train[df_train.tempo_no_emprego < media_tempo_emprego]



X1 = df_tempo_emprego.drop(columns = ['remuneracao', 'id', 'idade'], axis = 1)
y1 =df_tempo_emprego['remuneracao']

resultados = cross_validating_compare(X1, y1, pipe, ['Voting'], k=10, repeats=1, random_st=1)
resultados

,MAE TRAIN,MAE TRAIN STD,MAE TEST,MAE TEST STD
Voting,555.49,6.05,831.69,63.47


In [17]:
df_escolaridade = df_train[df_train.escolaridade == 7 ]

X1 = df_escolaridade.drop(columns = ['remuneracao', 'id', 'idade'], axis = 1)
y1 =df_escolaridade['remuneracao']

resultados = cross_validating_compare(X1, y1, pipe, ['Voting'], k=10, repeats=1, random_st=1)
resultados

,MAE TRAIN,MAE TRAIN STD,MAE TEST,MAE TEST STD
Voting,513.84,3.91,777.71,43.21


In [18]:
df_escolaridade = df_train[df_train.escolaridade < 7 ]

X1 = df_escolaridade.drop(columns = ['remuneracao', 'id', 'idade'], axis = 1)
y1 =df_escolaridade['remuneracao']

resultados = cross_validating_compare(X1, y1, pipe, ['Voting'], k=10, repeats=1, random_st=1)
resultados

,MAE TRAIN,MAE TRAIN STD,MAE TEST,MAE TEST STD
Voting,305.95,6.93,550.66,55.09


In [ ]:
df_cor_trab_semana = df_train[(df_train.horas_trabalho_por_semana > 35) & (df_train.horas_trabalho_por_semana != 40)]

X1 = df_cor_trab_semana.drop(columns = ['remuneracao', 'id', 'idade'], axis = 1)
y1 =df_cor_trab_semana['remuneracao']

resultados = cross_validating_compare(X1, y1, pipe, ['Voting'], k=10, repeats=1, random_st=1)
resultados

In [ ]:
df_eta = df_train[df_train.faixa_etaria < 4]

X1 = df_eta.drop(columns = ['remuneracao', 'id', 'idade'], axis = 1)
y1 =df_eta['remuneracao']

resultados = cross_validating_compare(X1, y1, pipe, ['Voting'], k=10, repeats=1, random_st=1)
resultados

In [ ]:
df_sexo = df_train[df_train.sexo == 2 ]

X = df_sexo.drop(columns = ['remuneracao', 'id', 'idade'], axis = 1)
y =df_sexo['remuneracao']


pipe1 = Pipeline(steps=[("preprocessor", preprocessor),('GradBoost', modelo_base)])

pipe = [TransformedTargetRegressor(regressor=pipe1,
                                   func=np.log, 
                                   inverse_func=np.exp)]

resultados = cross_validating_compare(X, y, pipe, ['log'], k=10, repeats=1, random_st = 1)
resultados

#### Conclusão: 
A avaliação de uma clusterização inicial dos dados não apresentou resultados positivos no modelo. Entretanto, notou-se que a análise de alguns grupos apresentaram uma melhora significativa no EMA.

OBS.: Cabe salientar que o tamanho da amostra dos dados deve possuir um número significativo para proceder com essa análise

Dessa forma, optou-se por substituir no modelo aqueles valores cujos grupos aprsentaram uma melhora no EMA.

# Alterando os preditos de acordo com uma 'clusterização especial'

In [ ]:
############################################################################
# Changing sexo == 2
df_train_sexo = df_train.copy()
df_train_sexo = df_train_sexo[df_train_sexo.sexo ==2]
df_testando_sexo = df_test.copy()


X_sexo = df_train_sexo.drop(columns = ['remuneracao', 'id', 'idade'], axis = 1)
y_sexo =df_train_sexo['remuneracao']


modelo_final_1.fit(X_sexo, y_sexo)

df_testando_sexo_ = df_testando_sexo.drop(columns = ['id', 'idade'], axis = 1)
preditos_final_sexo = modelo_final_1.predict(df_testando_sexo_)


df_testando_sexo['preditos'] = preditos_final_sexo

for indice, linha in df_testando.iterrows():
    if linha.sexo == 2:
        ident = int(linha.id)
        valor_predito = df_testando_sexo.loc[df_testando_sexo.id == ident, 'preditos']
        df_testando.loc[df_testando.id == ident, 'preditos'] = valor_predito

In [ ]:
#####################################################################
# empresa != 10

df_train_empresa = df_train.copy()
df_train_empresa = df_train_empresa[df_train_empresa.tamanho_da_empresa != 10]
df_testando_empresa = df_test.copy()


X = df_train_empresa.drop(columns = ['remuneracao', 'id', 'idade'], axis = 1)
y =df_train_empresa['remuneracao']


modelo_final_1.fit(X, y)

df_testando_empresa_ = df_testando_empresa.drop(columns = ['id', 'idade'], axis = 1)
preditos_final_empresa = modelo_final_1.predict(df_testando_empresa_)


df_testando_empresa['preditos'] = preditos_final_empresa

for indice, linha in df_testando.iterrows():
    if linha.tamanho_da_empresa != 10:
        ident = int(linha.id)
        valor_predito = df_testando_empresa.loc[df_testando_empresa.id == ident, 'preditos']
        df_testando.loc[df_testando.id == ident, 'preditos'] = valor_predito

In [ ]:
####################################################################
# cor != 99

df_train_cor = df_train.copy()
df_train_cor = df_train_cor[df_train_cor.cor_raca != 99]
df_testando_cor = df_test.copy()


X = df_train_cor.drop(columns = ['remuneracao', 'id', 'idade'], axis = 1)
y =df_train_cor['remuneracao']


modelo_final_1.fit(X, y)

df_testando_cor_ = df_testando_cor.drop(columns = ['id', 'idade'], axis = 1)
preditos_final_cor = modelo_final_1.predict(df_testando_cor_)


df_testando_cor['preditos'] = preditos_final_cor

for indice, linha in df_testando.iterrows():
    if linha.cor_raca != 99:
        ident = int(linha.id)
        valor_predito = df_testando_cor.loc[df_testando_cor.id == ident, 'preditos']
        df_testando.loc[df_testando.id == ident, 'preditos'] = valor_predito

In [ ]:
###############################################################################
# Emprego < média
media_tempo_emprego = df_train['tempo_no_emprego'].mean()

df_train_tempo_emprego = df_train.copy()
df_train_tempo_emprego = df_train_tempo_emprego[df_train_tempo_emprego.tempo_no_emprego < media_tempo_emprego]
df_testando_tempo_emprego = df_test.copy()


X_emprego = df_train_tempo_emprego.drop(columns = ['remuneracao', 'id', 'idade'], axis = 1)
y_emprego =df_train_tempo_emprego['remuneracao']


modelo_final_1.fit(X_emprego, y_emprego)

df_testando_tempo_emprego_ = df_testando_tempo_emprego.drop(columns = ['id', 'idade'], axis = 1)
preditos_final_emprego = modelo_final_1.predict(df_testando_tempo_emprego_)


df_testando_tempo_emprego['preditos'] = preditos_final_emprego

for indice, linha in df_testando.iterrows():
    if linha.tempo_no_emprego < media_tempo_emprego:
        ident = int(linha.id)
        valor_predito = df_testando_tempo_emprego.loc[df_testando_tempo_emprego.id == ident, 'preditos']
        df_testando.loc[df_testando.id == ident, 'preditos'] = valor_predito

In [ ]:
################################################
# Escolaridade = 7
df_train_escolar = df_train.copy()
df_train_escolar = df_train_escolar[df_train_escolar.escolaridade == 7]
df_testando_escolar = df_test.copy()


X = df_train_escolar.drop(columns = ['remuneracao', 'id', 'idade'], axis = 1)
y =df_train_escolar['remuneracao']


modelo_final_1.fit(X, y)

df_testando_escolar_ = df_testando_escolar.drop(columns = ['id', 'idade'], axis = 1)
preditos_final_escolar = modelo_final_1.predict(df_testando_escolar_)


df_testando_escolar['preditos'] = preditos_final_escolar

for indice, linha in df_testando.iterrows():
    if linha.escolaridade == 7:
        ident = int(linha.id)
        valor_predito = df_testando_escolar.loc[df_testando_escolar.id == ident, 'preditos']
        df_testando.loc[df_testando.id == ident, 'preditos'] = valor_predito

In [ ]:
#######################################################################
# horas de trabalho > 35 e != 40
df_train_horas = df_train.copy()
df_train_horas = df_train_horas[(df_train_horas.horas_trabalho_por_semana > 35) & (df_train_horas.horas_trabalho_por_semana != 40)]
df_testando_horas = df_test.copy()


X = df_train_horas.drop(columns = ['remuneracao', 'id', 'idade'], axis = 1)
y =df_train_horas['remuneracao']


modelo_final_1.fit(X, y)

df_testando_horas_ = df_testando_horas.drop(columns = ['id', 'idade'], axis = 1)
preditos_final_horas = modelo_final_1.predict(df_testando_horas_)


df_testando_horas['preditos'] = preditos_final_horas

for indice, linha in df_testando.iterrows():
    if (linha.horas_trabalho_por_semana > 35) and (linha.horas_trabalho_por_semana != 40):
        ident = int(linha.id)
        valor_predito = df_testando_horas.loc[df_testando_horas.id == ident, 'preditos']
        df_testando.loc[df_testando.id == ident, 'preditos'] = valor_predito

In [ ]:
################################################
# Escolaridae <7

df_train_escolar = df_train.copy()
df_train_escolar = df_train_escolar[df_train_escolar.escolaridade < 7]
df_testando_escolar = df_test.copy()


X = df_train_escolar.drop(columns = ['remuneracao', 'id', 'idade'], axis = 1)
y =df_train_escolar['remuneracao']


modelo_final_1.fit(X, y)

df_testando_escolar_ = df_testando_escolar.drop(columns = ['id', 'idade'], axis = 1)
preditos_final_escolar = modelo_final_1.predict(df_testando_escolar_)


df_testando_escolar['preditos'] = preditos_final_escolar

for indice, linha in df_testando.iterrows():
    if linha.escolaridade < 7:
        ident = int(linha.id)
        valor_predito = df_testando_escolar.loc[df_testando_escolar.id == ident, 'preditos']
        df_testando.loc[df_testando.id == ident, 'preditos'] = valor_predito

In [ ]:
#######################################
# faixa_etaria < 4

df_train_feta = df_train.copy()
df_train_feta = df_train_feta[df_train_feta.faixa_etaria < 4]
df_testando_feta = df_test.copy()


X = df_train_feta.drop(columns = ['remuneracao', 'id', 'idade'], axis = 1)
y =df_train_feta['remuneracao']


modelo_final_1.fit(X, y)

df_testando_feta_ = df_testando_feta.drop(columns = ['id', 'idade'], axis = 1)
preditos_final_feta = modelo_final_1.predict(df_testando_feta_)


df_testando_feta['preditos'] = preditos_final_feta


for indice, linha in df_testando.iterrows():
    if linha.faixa_etaria < 4:
        ident = int(linha.id)
        valor_predito = df_testando_feta.loc[df_testando_feta.id == ident, 'preditos']
        df_testando.loc[df_testando.id == ident, 'preditos'] = valor_predito

In [ ]:
# Duplicated Indexes

for indice, linha in df_treinando.iterrows():
    if linha.id in indices_duplicados:
        ident = int(linha.id)
        df_testando.loc[df_testando.id == ident, 'preditos'] = linha.remuneracao

# SUBMISSÃO 10

Nesta submissão o EMA atingiu o valor de 627.65, o que foi maior do que o esperado, indicando que a 'clusterização' deveria ser mais estudada. Entretando, não havia mais tempo para tal análise

In [ ]:
subm = pd.DataFrame()
subm['remuneracao'] = df_testando.preditos
subm.to_csv('submissão.csv', index = False)